<a href="https://colab.research.google.com/github/lpdata/fraude_bilhetagem/blob/main/notebooks/02_tratamento_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tratamento de Features

## 0. Contexto e objetivo

Este notebook tem como objetivo consolidar o tratamento dos dados e a engenharia de features a partir do conjunto explorado na etapa anterior. Após a compreensão da estrutura dos dados, dos padrões comportamentais e dos principais sinais associados à ocorrência de fraude, esta etapa é responsável por transformar essas evidências exploratórias em atributos estruturados, reprodutíveis e adequados para a modelagem preditiva.

As atividades realizadas aqui incluem a padronização de tipos e schema, o tratamento de inconsistências e valores inválidos, bem como a criação de features temporais, comportamentais e operacionais fundamentadas nas análises exploratórias previamente conduzidas. Todas as transformações são aplicadas de forma determinística e sem utilização da variável alvo como insumo, garantindo a mitigação de vazamento de informação.

Ao final deste notebook, é gerado um dataset processado e documentado, pronto para ser utilizado na etapa de modelagem, preservando rastreabilidade, clareza metodológica e alinhamento com as boas práticas de projetos de ciência de dados aplicados à detecção de fraude.

## 1. Vinculação com Github

In [149]:
!git clone https://github.com/lpdata/fraude_bilhetagem

Cloning into 'fraude_bilhetagem'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 111 (delta 53), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (111/111), 3.27 MiB | 5.52 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [150]:
%cd fraude_bilhetagem

/content/fraude_bilhetagem/fraude_bilhetagem/fraude_bilhetagem


In [151]:
!ls

data  notebooks  README.md  requirements.txt  src


In [152]:
!ls data/raw

dados.csv


In [153]:
import os
print("Diretório atual:", os.getcwd()) #checando se estou no diretório correto

Diretório atual: /content/fraude_bilhetagem/fraude_bilhetagem/fraude_bilhetagem


## 2. Imports e configurações

In [154]:
import pandas as pd
import numpy as np

import os
import sys
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')


In [155]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.float_format', '{:.4f}'.format)


In [156]:
RANDOM_STATE = 42 #Seed para reprodutibilidade
np.random.seed(RANDOM_STATE)


In [157]:
# Definição dos diretórios do projeto

BASE_DIR = Path.cwd()

DATA_DIR = BASE_DIR / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
OUTPUTS_DIR = BASE_DIR / 'outputs'

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)


In [158]:
print("Diretório base:", BASE_DIR)
print("Arquivos em data/raw:", list(RAW_DIR.iterdir()))


Diretório base: /content/fraude_bilhetagem/fraude_bilhetagem/fraude_bilhetagem
Arquivos em data/raw: [PosixPath('/content/fraude_bilhetagem/fraude_bilhetagem/fraude_bilhetagem/data/raw/.gitkeep'), PosixPath('/content/fraude_bilhetagem/fraude_bilhetagem/fraude_bilhetagem/data/raw/dados.csv')]


In [159]:
def checkpoint(msg):
    print(f"✔ {msg}")

checkpoint("Etapa 2 concluída: imports, configurações e diretórios prontos")

✔ Etapa 2 concluída: imports, configurações e diretórios prontos


<small>***Comentários Letícia:** Nesta etapa, realizei a configuração inicial do ambiente de trabalho, incluindo a importação das bibliotecas necessárias, a definição de parâmetros globais e a validação da estrutura do projeto. Essas configurações garantem reprodutibilidade, organização do código e consistência nas etapas subsequentes de tratamento e engenharia de features.*</small>

## 3. Carregamento dos dados e validação de integridade

### 3.1 Carregar o dataset raw

In [160]:
DATA_PATH = RAW_DIR / "dados.csv"

df = pd.read_csv(DATA_PATH)

checkpoint(f"Dados carregados com sucesso: {DATA_PATH}")
checkpoint(f"Shape do dataset: {df.shape}")
display(df.head())


✔ Dados carregados com sucesso: /content/fraude_bilhetagem/fraude_bilhetagem/fraude_bilhetagem/data/raw/dados.csv
✔ Shape do dataset: (30000, 28)


,id_transacao,id_cartao,ts_transacao,dt_emissao_cartao,nu_recargas_acumulado,valor_transacao,tipo_cartao,status_cartao,id_dispositivo,id_motorista,linha_onibus,sentido_viagem,clima_dia,temp_externa,id_loja_ultima_recarga,tipo_pagamento_recarga,latitude,longitude,cidade,feriado,integracao_metro,limite_diario_uso,idade_usuario,sexo_usuario,diff_tempo_segundos,tempo_vida_cartao_dias,target_fraude,bloqueio_automatico_sistema
0,160,18901,2026-01-21 20:23:29,2022-10-17,27,4.5000,Funcionario,Ativo,560,1155,88,Volta,Chuva,24.8087,15,Cartao_Credito,-23.5194,-46.6360,São Paulo,0,0,10,26,N,754063.0000,1192,0,0
1,103,18392,2026-01-29 03:13:36,2023-06-30,142,4.5000,Idoso,Ativo,597,1184,99,Volta,Chuva,26.5893,44,Dinheiro,-23.5968,-46.6113,São Paulo,0,1,10,39,F,1050052.0000,944,0,0
2,22320,17050,2026-01-04 16:05:52,2020-10-29,51,4.5000,Comum,Ativo,555,1118,7,Volta,Chuva,37.0170,13,PIX,-23.5743,-46.6591,São Paulo,0,0,10,45,F,25224.0000,1893,0,0
3,12932,11516,2026-01-15 01:29:07,2023-09-23,34,9.0000,Idoso,Ativo,500,1152,129,Volta,Chuva,29.3219,26,PIX,-23.5572,-46.6581,São Paulo,0,1,10,20,N,NaN,845,0,0
4,21,16396,2026-01-19 00:20:30,2023-01-10,78,4.5000,Vale-Transporte,Ativo,563,1078,111,Ida,Chuva,24.1620,12,Cartao_Credito,-23.5435,-46.6858,São Paulo,0,0,10,45,M,507280.0000,1105,0,0


### 3.2 Visão geral rápida (colunas e tipos atuais)

In [161]:
checkpoint("Visão geral do dataset (dtypes e nulos iniciais)")

display(df.dtypes)
display(df.isna().mean().sort_values(ascending=False).head(15))


✔ Visão geral do dataset (dtypes e nulos iniciais)


,0
id_transacao,int64
id_cartao,int64
ts_transacao,object
dt_emissao_cartao,object
nu_recargas_acumulado,int64
valor_transacao,float64
tipo_cartao,object
status_cartao,object
id_dispositivo,int64
id_motorista,int64


,0
diff_tempo_segundos,0.3161
id_transacao,0.0000
ts_transacao,0.0000
id_cartao,0.0000
nu_recargas_acumulado,0.0000
valor_transacao,0.0000
tipo_cartao,0.0000
status_cartao,0.0000
id_dispositivo,0.0000
id_motorista,0.0000


### 3.3 Validação de schema (colunas esperadas)

In [162]:
expected_cols = [
    "id_transacao","id_cartao","ts_transacao","dt_emissao_cartao","nu_recargas_acumulado",
    "valor_transacao","tipo_cartao","status_cartao","id_dispositivo","id_motorista",
    "linha_onibus","sentido_viagem","clima_dia","temp_externa","id_loja_ultima_recarga",
    "tipo_pagamento_recarga","latitude","longitude","cidade","feriado","integracao_metro",
    "limite_diario_uso","idade_usuario","sexo_usuario","bloqueio_auto_sistema","target_fraude"
]

missing_cols = [c for c in expected_cols if c not in df.columns]
extra_cols = [c for c in df.columns if c not in expected_cols]

if missing_cols:
    print("⚠️ Colunas esperadas ausentes:", missing_cols)
else:
    checkpoint("Nenhuma coluna esperada está ausente.")

if extra_cols:
    print("ℹ️ Colunas extras encontradas (não previstas no dicionário):", extra_cols)
else:
    checkpoint("Nenhuma coluna extra encontrada.")


⚠️ Colunas esperadas ausentes: ['bloqueio_auto_sistema']
ℹ️ Colunas extras encontradas (não previstas no dicionário): ['diff_tempo_segundos', 'tempo_vida_cartao_dias', 'bloqueio_automatico_sistema']


### 3.4 Checagem de duplicidade (linhas e chaves)

In [163]:
dup_rows = df.duplicated().sum()
dup_id_transacao = df["id_transacao"].duplicated().sum()

checkpoint(f"Linhas duplicadas completas: {dup_rows}")
checkpoint(f"id_transacao duplicado: {dup_id_transacao}")


✔ Linhas duplicadas completas: 0
✔ id_transacao duplicado: 0


### 3.5 Checagem de valores básicos (sanity checks)

In [164]:
checkpoint("Sanity checks básicos")

# Contagem de valores únicos em chaves
print("Transações únicas:", df["id_transacao"].nunique())
print("Cartões únicos:", df["id_cartao"].nunique())

# Target: distribuição inicial
print("\nDistribuição do target_fraude:")
display(df["target_fraude"].value_counts(dropna=False))


✔ Sanity checks básicos
Transações únicas: 30000
Cartões únicos: 9483

Distribuição do target_fraude:


,count
target_fraude,
0,27065
1,2935


### 3.6 Checkpoint final da etapa 3

In [165]:
checkpoint("Etapa 3 concluída: dados carregados e integridade inicial verificada")


✔ Etapa 3 concluída: dados carregados e integridade inicial verificada


<small>***Comentários Letícia:** Nesta etapa, validei a integridade estrutural do dataset carregado, identificando divergências de schema em relação ao dicionário de dados e a presença de colunas derivadas não previstas no conjunto bruto. Os dados não apresentaram duplicidades nem inconsistências críticas de chave, e a distribuição da variável alvo confirmou um cenário de desbalanceamento moderado. Esses achados orientam diretamente as decisões de padronização e tratamento a serem realizadas nas etapas subsequentes.*</small>

## 4.Padronização de tipos e parsing (schema enforcement)

### 4.1 Parsing de ts_transacao e dt_emissao_cartao

In [166]:
# Parsing de colunas temporais
df['ts_transacao'] = pd.to_datetime(df['ts_transacao'], errors='coerce')
df['dt_emissao_cartao'] = pd.to_datetime(df['dt_emissao_cartao'], errors='coerce')

checkpoint("Parsing de datas concluído: ts_transacao e dt_emissao_cartao convertidas para datetime")


✔ Parsing de datas concluído: ts_transacao e dt_emissao_cartao convertidas para datetime


In [167]:
# Verificação dos tipos após conversão
display(df[['ts_transacao', 'dt_emissao_cartao']].dtypes)

# Checagem de valores nulos gerados no parsing
display(
    df[['ts_transacao', 'dt_emissao_cartao']]
    .isna()
    .mean()
    .mul(100)
    .round(2)
)


,0
ts_transacao,datetime64[ns]
dt_emissao_cartao,datetime64[ns]


,0
ts_transacao,0.0000
dt_emissao_cartao,0.0000


<small>***Comentários Letícia:** A conversão das colunas temporais foi realizada com sucesso, sem geração de valores ausentes, indicando consistência e boa qualidade dos dados de data e hora. Com isso, o dataset está apto para operações temporais e para a criação de features dependentes de tempo nas etapas seguintes.*</small>

### 4.2 Normalização de binárias

In [168]:
# Renomear coluna para alinhar com o dicionário de dados
df = df.rename(columns={'bloqueio_automatico_sistema': 'bloqueio_auto_sistema'})

checkpoint("Coluna 'bloqueio_automatico_sistema' renomeada para 'bloqueio_auto_sistema'")

# Função utilitária para normalização binária
def normalize_binary(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.strip().str.lower()
    mapping = {
        '1': 1, 'true': 1, 'sim': 1, 's': 1,
        '0': 0, 'false': 0, 'nao': 0, 'não': 0, 'n': 0
    }
    return s.map(mapping)

# Aplicar normalização
binary_cols = [
    'feriado',
    'integracao_metro',
    'bloqueio_auto_sistema',
    'target_fraude'
]

for col in binary_cols:
    df[col] = normalize_binary(df[col]).astype('Int64')

checkpoint("Variáveis binárias normalizadas para 0/1")

✔ Coluna 'bloqueio_automatico_sistema' renomeada para 'bloqueio_auto_sistema'
✔ Variáveis binárias normalizadas para 0/1


In [169]:
display(df[binary_cols].dtypes)
display(df[binary_cols].value_counts(dropna=False))


,0
feriado,Int64
integracao_metro,Int64
bloqueio_auto_sistema,Int64
target_fraude,Int64


feriado  integracao_metro  bloqueio_auto_sistema  target_fraude
0        0                 0                      0                12874
         1                 0                      0                12831
                           1                      1                 1421
         0                 1                      1                 1380
1        0                 0                      0                  687
         1                 0                      0                  673
                           1                      1                   73
         0                 1                      1                   61
Name: count, dtype: int64

<small>***Comentários Letícia:** As variáveis binárias foram normalizadas para o formato 0/1 e tiveram sua nomenclatura alinhada ao dicionário de dados. A validação posterior confirmou a ausência de valores inconsistentes, assegurando padronização semântica e compatibilidade com as etapas subsequentes de tratamento e modelagem.*</small>

### 4.3 Coerção de numéricas

In [170]:
numeric_cols = [
    'id_transacao',
    'id_cartao',
    'nu_recargas_acumulado',
    'valor_transacao',
    'id_dispositivo',
    'id_motorista',
    'linha_onibus',
    'temp_externa',
    'id_loja_ultima_recarga',
    'limite_diario_uso',
    'idade_usuario'
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

checkpoint("Coerção explícita aplicada às colunas numéricas")


✔ Coerção explícita aplicada às colunas numéricas


In [171]:
int_cols = [
    'id_transacao',
    'id_cartao',
    'nu_recargas_acumulado',
    'id_dispositivo',
    'id_motorista',
    'linha_onibus',
    'id_loja_ultima_recarga',
    'limite_diario_uso',
    'idade_usuario'
]

float_cols = [
    'valor_transacao',
    'temp_externa'
]

for col in int_cols:
    df[col] = df[col].astype('Int64')

for col in float_cols:
    df[col] = df[col].astype(float)

checkpoint("Tipos numéricos padronizados (inteiros e contínuos)")


✔ Tipos numéricos padronizados (inteiros e contínuos)


In [172]:
display(df[int_cols + float_cols].dtypes)

display(
    df[int_cols + float_cols]
    .isna()
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

checkpoint("Item 4.3 concluído: variáveis numéricas coercidas e padronizadas")

,0
id_transacao,Int64
id_cartao,Int64
nu_recargas_acumulado,Int64
id_dispositivo,Int64
id_motorista,Int64
linha_onibus,Int64
id_loja_ultima_recarga,Int64
limite_diario_uso,Int64
idade_usuario,Int64
valor_transacao,float64


,0
id_transacao,0.0000
id_cartao,0.0000
nu_recargas_acumulado,0.0000
id_dispositivo,0.0000
id_motorista,0.0000
linha_onibus,0.0000
id_loja_ultima_recarga,0.0000
limite_diario_uso,0.0000
idade_usuario,0.0000
valor_transacao,0.0000


✔ Item 4.3 concluído: variáveis numéricas coercidas e padronizadas


<small>***Comentários Letícia:** As variáveis numéricas foram explicitamente coercidas e padronizadas quanto ao tipo, garantindo separação adequada entre identificadores, contadores e variáveis contínuas. A validação posterior confirmou a ausência de valores inválidos ou ausentes introduzidos nesta etapa, indicando consistência numérica do conjunto de dados e preparando a base para as próximas fases de tratamento e engenharia de features.*</small>

### 4.4 Padronização de categóricas

In [173]:
categorical_cols = [
    'tipo_cartao',
    'status_cartao',
    'sentido_viagem',
    'clima_dia',
    'tipo_pagamento_recarga',
    'cidade',
    'sexo_usuario'
]

for col in categorical_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.lower()
        .replace({'nan': pd.NA})
    )

checkpoint("Padronização básica aplicada às variáveis categóricas (lowercase + trim)")

✔ Padronização básica aplicada às variáveis categóricas (lowercase + trim)


In [174]:
for col in categorical_cols:
    df[col] = df[col].astype('category')

checkpoint("Variáveis categóricas convertidas para dtype 'category'")


✔ Variáveis categóricas convertidas para dtype 'category'


In [175]:
display(df[categorical_cols].dtypes)

# Cardinalidade por coluna
for col in categorical_cols:
    print(f"\n{col} - valores únicos:")
    print(df[col].value_counts(dropna=False).head(10))

checkpoint("Item 4.4 concluído: variáveis categóricas padronizadas")

,0
tipo_cartao,category
status_cartao,category
sentido_viagem,category
clima_dia,category
tipo_pagamento_recarga,category
cidade,category
sexo_usuario,category



tipo_cartao - valores únicos:
tipo_cartao
comum              6035
funcionario        6012
idoso              6006
estudante          5982
vale-transporte    5965
Name: count, dtype: int64

status_cartao - valores únicos:
status_cartao
ativo        28216
suspenso       920
bloqueado      864
Name: count, dtype: int64

sentido_viagem - valores únicos:
sentido_viagem
ida      15065
volta    14935
Name: count, dtype: int64

clima_dia - valores únicos:
clima_dia
chuva         10063
ensolarado    10003
nublado        9934
Name: count, dtype: int64

tipo_pagamento_recarga - valores únicos:
tipo_pagamento_recarga
pix               10149
cartao_credito     9957
dinheiro           9894
Name: count, dtype: int64

cidade - valores únicos:
cidade
são paulo    30000
Name: count, dtype: int64

sexo_usuario - valores únicos:
sexo_usuario
f    10019
m    10006
n     9975
Name: count, dtype: int64
✔ Item 4.4 concluído: variáveis categóricas padronizadas


<small>***Comentários Letícia:** As variáveis categóricas foram padronizadas quanto à grafia e convertidas para o tipo category, assegurando consistência semântica e reduzindo ruído nos dados. A validação posterior indicou categorias bem definidas e distribuição equilibrada na maioria das variáveis, com destaque para a coluna cidade, que apresentou ausência de variabilidade e deverá ser reavaliada quanto à sua utilidade nas próximas etapas do pipeline.*</small>



### 4.5 Checkpoint de tipos finais

In [176]:
checkpoint("Visão consolidada do schema após padronização")
display(df.dtypes)


✔ Visão consolidada do schema após padronização


,0
id_transacao,Int64
id_cartao,Int64
ts_transacao,datetime64[ns]
dt_emissao_cartao,datetime64[ns]
nu_recargas_acumulado,Int64
valor_transacao,float64
tipo_cartao,category
status_cartao,category
id_dispositivo,Int64
id_motorista,Int64


In [177]:
checkpoint("Percentual de valores ausentes por coluna (top 15)")
display(
    df.isna()
      .mean()
      .mul(100)
      .round(2)
      .sort_values(ascending=False)
      .head(15)
)


✔ Percentual de valores ausentes por coluna (top 15)


,0
diff_tempo_segundos,31.6100
id_transacao,0.0000
ts_transacao,0.0000
id_cartao,0.0000
nu_recargas_acumulado,0.0000
valor_transacao,0.0000
tipo_cartao,0.0000
status_cartao,0.0000
id_dispositivo,0.0000
id_motorista,0.0000


In [178]:
expected_schema = [
    "id_transacao","id_cartao","ts_transacao","dt_emissao_cartao",
    "nu_recargas_acumulado","valor_transacao","tipo_cartao","status_cartao",
    "id_dispositivo","id_motorista","linha_onibus","sentido_viagem",
    "clima_dia","temp_externa","id_loja_ultima_recarga","tipo_pagamento_recarga",
    "latitude","longitude","cidade","feriado","integracao_metro",
    "limite_diario_uso","idade_usuario","sexo_usuario",
    "bloqueio_auto_sistema","target_fraude"
]

extra_cols = [c for c in df.columns if c not in expected_schema]
missing_cols = [c for c in expected_schema if c not in df.columns]

if extra_cols:
    print("⚠️ Colunas fora do schema esperado:", extra_cols)
else:
    checkpoint("Nenhuma coluna extra fora do schema esperado.")

if missing_cols:
    print("⚠️ Colunas esperadas ausentes:", missing_cols)
else:
    checkpoint("Nenhuma coluna esperada ausente.")

checkpoint("Etapa 4 concluída: schema padronizado e validado")

⚠️ Colunas fora do schema esperado: ['diff_tempo_segundos', 'tempo_vida_cartao_dias']
✔ Nenhuma coluna esperada ausente.
✔ Etapa 4 concluída: schema padronizado e validado


<small>***Comentários Letícia:** Ao final desta etapa, o schema do dataset foi padronizado e validado com sucesso, contemplando a conversão adequada de tipos temporais, numéricos, categóricos e binários. A validação final confirmou o alinhamento com o dicionário de dados, bem como a presença de colunas derivadas não previstas no conjunto bruto, que serão tratadas na próxima etapa. Com isso, a base encontra-se estruturalmente consistente e pronta para o tratamento de qualidade e engenharia de features.*</small>

## 5. Tratamento de inconsistências e qualidade dos dados

### 5.1 Remoção de duplicatas completas

In [179]:
# Remoção das colunas

cols_to_drop = [
    'diff_tempo_segundos',
    'tempo_vida_cartao_dias'
]

df = df.drop(columns=cols_to_drop, errors='ignore')

checkpoint(f"Colunas removidas do dataset base: {cols_to_drop}")

✔ Colunas removidas do dataset base: ['diff_tempo_segundos', 'tempo_vida_cartao_dias']


In [180]:
checkpoint("Validação do schema após remoção de colunas indevidas")

display(df.columns)

checkpoint("Item 5.1 concluído: colunas fora do schema removidas")

✔ Validação do schema após remoção de colunas indevidas


Index(['id_transacao', 'id_cartao', 'ts_transacao', 'dt_emissao_cartao',
       'nu_recargas_acumulado', 'valor_transacao', 'tipo_cartao',
       'status_cartao', 'id_dispositivo', 'id_motorista', 'linha_onibus',
       'sentido_viagem', 'clima_dia', 'temp_externa', 'id_loja_ultima_recarga',
       'tipo_pagamento_recarga', 'latitude', 'longitude', 'cidade', 'feriado',
       'integracao_metro', 'limite_diario_uso', 'idade_usuario',
       'sexo_usuario', 'target_fraude', 'bloqueio_auto_sistema'],
      dtype='object')

✔ Item 5.1 concluído: colunas fora do schema removidas


<small>***Comentários Letícia:** Neste passo, removi do dataset base as colunas derivadas não previstas no dicionário de dados, previamente identificadas como inconsistências estruturais. Com isso, o conjunto de dados passa a refletir fielmente o dado bruto padronizado, preservando a integridade do schema e evitando riscos de vazamento de informação nas etapas subsequentes.*</small>

### 5.2 Regra para id_transacao duplicado (decisão e aplicação)

In [181]:
# Verificação de duplicidade de linhas completas
dup_linhas = df.duplicated().sum()

checkpoint(f"Linhas duplicadas completas identificadas: {dup_linhas}")

✔ Linhas duplicadas completas identificadas: 0


In [182]:
# Verificação de duplicidade da chave primária (id_transacao)
dup_transacao = df['id_transacao'].duplicated().sum()

checkpoint(f"Duplicidades na chave id_transacao: {dup_transacao}")

✔ Duplicidades na chave id_transacao: 0


In [183]:
# Verificação de cardinalidade das chaves
checkpoint("Validação de cardinalidade das principais chaves")

print("Total de registros:", len(df))
print("Transações únicas:", df['id_transacao'].nunique())
print("Cartões únicos:", df['id_cartao'].nunique())

✔ Validação de cardinalidade das principais chaves
Total de registros: 30000
Transações únicas: 30000
Cartões únicos: 9483


In [184]:
# Validação cruzada simples (consistência transação × cartão)
inconsistencias_cartao = (
    df.groupby('id_transacao')['id_cartao']
      .nunique()
      .gt(1)
      .sum()
)

checkpoint(f"Transações associadas a mais de um cartão: {inconsistencias_cartao}")
checkpoint("Item 5.2 concluído: duplicidades e chaves validadas")

✔ Transações associadas a mais de um cartão: 0
✔ Item 5.2 concluído: duplicidades e chaves validadas


<small>***Comentários Letícia:** A verificação de duplicidades e integridade das chaves confirmou que o conjunto de dados não apresenta registros duplicados nem inconsistências na identificação das transações e dos cartões. Essa validação assegura a confiabilidade da base para operações de agregação, análise temporal e engenharia de features nas etapas subsequentes.*</small>



### 5.3 Tratamento de valores inválidos

In [185]:
# Validação de faixas plausíveis (sanity checks de domínio)

checkpoint("Validação de faixas plausíveis – sanity checks")

sanity_checks = {
    'idade_usuario': (0, 120),
    'valor_transacao': (0, None),
    'temp_externa': (-20, 60),
    'latitude': (-90, 90),
    'longitude': (-180, 180)
}

for col, (min_val, max_val) in sanity_checks.items():
    if max_val is not None:
        invalid = df[(df[col] < min_val) | (df[col] > max_val)][col].count()
    else:
        invalid = df[df[col] < min_val][col].count()

    print(f"{col}: valores inválidos identificados = {invalid}")

✔ Validação de faixas plausíveis – sanity checks
idade_usuario: valores inválidos identificados = 57
valor_transacao: valores inválidos identificados = 0
temp_externa: valores inválidos identificados = 0
latitude: valores inválidos identificados = 0
longitude: valores inválidos identificados = 0


In [186]:
# Tratamento conservador de valores inválidos

for col, (min_val, max_val) in sanity_checks.items():
    if max_val is not None:
        df.loc[(df[col] < min_val) | (df[col] > max_val), col] = pd.NA
    else:
        df.loc[df[col] < min_val, col] = pd.NA

checkpoint("Valores fora de faixa tratados como ausentes (NaN)")

✔ Valores fora de faixa tratados como ausentes (NaN)


In [187]:
# Validação pós-tratamento

checkpoint("Percentual de valores ausentes após tratamento de inconsistências")

display(
    df[list(sanity_checks.keys())]
      .isna()
      .mean()
      .mul(100)
      .round(2)
)

checkpoint("Item 5.3 concluído: valores inválidos tratados de forma conservadora")

✔ Percentual de valores ausentes após tratamento de inconsistências


,0
idade_usuario,0.1900
valor_transacao,0.0000
temp_externa,0.0000
latitude,0.0000
longitude,0.0000


✔ Item 5.3 concluído: valores inválidos tratados de forma conservadora


<small>***Comentários Letícia:** A validação de faixas plausíveis indicou inconsistências pontuais apenas na variável idade do usuário, representando uma fração muito pequena do conjunto de dados. Esses valores foram tratados de forma conservadora, sendo convertidos em ausentes, sem exclusão de registros ou imputações arbitrárias. O impacto do saneamento foi mínimo, preservando a qualidade e a integridade do dataset para as próximas etapas.*</small>

### 5.4 Análise de nulos

In [188]:
# Identificação de variáveis constantes

checkpoint("Identificação de variáveis com variância zero")

constantes = [
    col for col in df.columns
    if df[col].nunique(dropna=False) <= 1
]

print("Variáveis constantes identificadas:", constantes)

✔ Identificação de variáveis com variância zero
Variáveis constantes identificadas: ['cidade', 'limite_diario_uso']


In [189]:
# Remoção das variáveis sem variabilidade

df = df.drop(columns=constantes, errors='ignore')

checkpoint(f"Variáveis sem poder informativo removidas: {constantes}")

✔ Variáveis sem poder informativo removidas: ['cidade', 'limite_diario_uso']


In [190]:
# Validação pós-remoção

checkpoint("Validação do schema após remoção de variáveis constantes")
display(df.columns)

checkpoint("Item 5.4 concluído: variáveis sem variância removidas")

✔ Validação do schema após remoção de variáveis constantes


Index(['id_transacao', 'id_cartao', 'ts_transacao', 'dt_emissao_cartao',
       'nu_recargas_acumulado', 'valor_transacao', 'tipo_cartao',
       'status_cartao', 'id_dispositivo', 'id_motorista', 'linha_onibus',
       'sentido_viagem', 'clima_dia', 'temp_externa', 'id_loja_ultima_recarga',
       'tipo_pagamento_recarga', 'latitude', 'longitude', 'feriado',
       'integracao_metro', 'idade_usuario', 'sexo_usuario', 'target_fraude',
       'bloqueio_auto_sistema'],
      dtype='object')

✔ Item 5.4 concluído: variáveis sem variância removidas


<small>***Comentários Letícia:** A análise de variabilidade identificou colunas que apresentavam valor constante em todo o conjunto de dados, não oferecendo qualquer poder discriminativo para a identificação de fraude. As variáveis cidade e limite_diario_uso foram removidas, reduzindo ruído e complexidade do dataset, sem impacto negativo na informação relevante para as próximas etapas do pipeline.*</small>

### 5.5 Tratamento mínimo de nulos críticos


In [191]:
# Percentual final de valores ausentes por coluna

checkpoint("Percentual final de valores ausentes por coluna")

display(
    df.isna()
      .mean()
      .mul(100)
      .round(2)
      .sort_values(ascending=False)
)

✔ Percentual final de valores ausentes por coluna


,0
idade_usuario,0.1900
id_transacao,0.0000
ts_transacao,0.0000
id_cartao,0.0000
nu_recargas_acumulado,0.0000
valor_transacao,0.0000
tipo_cartao,0.0000
dt_emissao_cartao,0.0000
status_cartao,0.0000
id_dispositivo,0.0000


In [192]:
# Visão resumida das colunas com valores ausentes

cols_com_na = df.columns[df.isna().any()].tolist()

print("Colunas com valores ausentes:", cols_com_na)

Colunas com valores ausentes: ['idade_usuario']


In [193]:
# Avaliação do impacto dos valores ausentes

checkpoint("Impacto dos valores ausentes no dataset")

total_linhas = len(df)
linhas_com_na = df.isna().any(axis=1).sum()

print(f"Total de registros: {total_linhas}")
print(f"Registros com pelo menos um valor ausente: {linhas_com_na}")
print(f"Percentual de registros afetados: {linhas_com_na / total_linhas * 100:.2f}%")

checkpoint("Item 5.5 concluído: valores ausentes analisados e consolidados")

✔ Impacto dos valores ausentes no dataset
Total de registros: 30000
Registros com pelo menos um valor ausente: 57
Percentual de registros afetados: 0.19%
✔ Item 5.5 concluído: valores ausentes analisados e consolidados


<small>***Comentários Letícia:** A análise final de valores ausentes indicou impacto mínimo no conjunto de dados, restrito a uma pequena parcela de registros na variável idade do usuário. Considerando a baixa representatividade desses casos e sua origem conhecida, optei por não realizar imputações ou exclusões nesta etapa, mantendo o dataset íntegro e preservando decisões para fases posteriores, onde o contexto analítico poderá ser melhor avaliado.*</small>

### 5.6 Checkpoint de qualidade pós-tratamento

In [194]:
checkpoint("Checkpoint final de qualidade pós-tratamento")

# Visão geral do dataset
print("Shape final do dataset:", df.shape)
display(df.head())

# Métricas de integridade
print("\nResumo de integridade:")
print("Registros totais:", len(df))
print("Transações únicas:", df['id_transacao'].nunique())
print("Cartões únicos:", df['id_cartao'].nunique())

# Valores ausentes
perc_linhas_na = df.isna().any(axis=1).mean() * 100
print(f"\nPercentual de registros com ao menos um valor ausente: {perc_linhas_na:.2f}%")
print("Colunas com valores ausentes:", df.columns[df.isna().any()].tolist())

# Duplicidades
print("\nChecagem de duplicidades:")
print("Linhas duplicadas completas:", df.duplicated().sum())
print("Duplicatas em id_transacao:", df['id_transacao'].duplicated().sum())

checkpoint("Etapa 5 concluída: qualidade dos dados validada e consolidada")

✔ Checkpoint final de qualidade pós-tratamento
Shape final do dataset: (30000, 24)


,id_transacao,id_cartao,ts_transacao,dt_emissao_cartao,nu_recargas_acumulado,valor_transacao,tipo_cartao,status_cartao,id_dispositivo,id_motorista,linha_onibus,sentido_viagem,clima_dia,temp_externa,id_loja_ultima_recarga,tipo_pagamento_recarga,latitude,longitude,feriado,integracao_metro,idade_usuario,sexo_usuario,target_fraude,bloqueio_auto_sistema
0,160,18901,2026-01-21 20:23:29,2022-10-17,27,4.5000,funcionario,ativo,560,1155,88,volta,chuva,24.8087,15,cartao_credito,-23.5194,-46.6360,0,0,26,n,0,0
1,103,18392,2026-01-29 03:13:36,2023-06-30,142,4.5000,idoso,ativo,597,1184,99,volta,chuva,26.5893,44,dinheiro,-23.5968,-46.6113,0,1,39,f,0,0
2,22320,17050,2026-01-04 16:05:52,2020-10-29,51,4.5000,comum,ativo,555,1118,7,volta,chuva,37.0170,13,pix,-23.5743,-46.6591,0,0,45,f,0,0
3,12932,11516,2026-01-15 01:29:07,2023-09-23,34,9.0000,idoso,ativo,500,1152,129,volta,chuva,29.3219,26,pix,-23.5572,-46.6581,0,1,20,n,0,0
4,21,16396,2026-01-19 00:20:30,2023-01-10,78,4.5000,vale-transporte,ativo,563,1078,111,ida,chuva,24.1620,12,cartao_credito,-23.5435,-46.6858,0,0,45,m,0,0



Resumo de integridade:
Registros totais: 30000
Transações únicas: 30000
Cartões únicos: 9483

Percentual de registros com ao menos um valor ausente: 0.19%
Colunas com valores ausentes: ['idade_usuario']

Checagem de duplicidades:
Linhas duplicadas completas: 0
Duplicatas em id_transacao: 0
✔ Etapa 5 concluída: qualidade dos dados validada e consolidada


<small>***Comentários Letícia:** Ao final desta etapa, o conjunto de dados apresentou elevada qualidade estrutural e semântica, com integridade total das chaves, ausência de duplicidades e impacto mínimo de valores ausentes. As inconsistências identificadas foram tratadas de forma conservadora, preservando a informação relevante e evitando decisões arbitrárias. Com isso, o dataset encontra-se consolidado, limpo e confiável, estando apto para a etapa de engenharia de features e posterior modelagem preditiva.*</small>

## 6. Engenharia de features

### 6.1 Ordenação base

In [195]:
checkpoint("Início da Etapa 6: preparação para engenharia de features")

# Garantia de ordenação para features temporais e comportamentais
df = df.sort_values(['id_cartao', 'ts_transacao']).reset_index(drop=True)

# Validação rápida da ordenação (amostra de um cartão)
cartao_exemplo = df['id_cartao'].iloc[0]
amostra = df[df['id_cartao'] == cartao_exemplo][['id_cartao', 'ts_transacao']].head(10)

print(f"Amostra de ordenação para id_cartao = {cartao_exemplo}:")
display(amostra)

checkpoint("Dataset ordenado por id_cartao e ts_transacao (pronto para criação de features)")

✔ Início da Etapa 6: preparação para engenharia de features
Amostra de ordenação para id_cartao = 10000:


,id_cartao,ts_transacao
0,10000,2026-01-24 10:54:15
1,10000,2026-01-29 10:03:32


✔ Dataset ordenado por id_cartao e ts_transacao (pronto para criação de features)


<small>***Comentários Letícia:** Antes da criação das novas features, organizei o conjunto de dados por cartão e ordem temporal das transações. Essa preparação garante que os atributos derivados de sequência e comportamento reflitam corretamente o histórico real de uso de cada cartão, evitando inconsistências nos cálculos temporais.*</small>

### 6.2 Features temporais básicas

In [196]:
# Hora da transação (0–23)
df['hora_transacao'] = df['ts_transacao'].dt.hour

# Dia da semana (0 = segunda, 6 = domingo)
df['dia_semana'] = df['ts_transacao'].dt.dayofweek

# Data da transação (apenas a data, sem horário)
df['data_transacao'] = df['ts_transacao'].dt.date

# Indicador de fim de semana
df['fim_de_semana'] = df['dia_semana'].isin([5, 6]).astype(int)

# Tempo de vida do cartão em dias no momento da transação
df['tempo_vida_cartao_dias'] = (
    (df['ts_transacao'].dt.floor('D') - df['dt_emissao_cartao'].dt.floor('D'))
    .dt.days
)

checkpoint(
    "Features temporais básicas criadas: "
    "hora_transacao, dia_semana, data_transacao, fim_de_semana, tempo_vida_cartao_dias"
)

display(
    df[['hora_transacao', 'dia_semana', 'fim_de_semana', 'tempo_vida_cartao_dias']]
    .describe()
)

display(df[['ts_transacao', 'dt_emissao_cartao', 'data_transacao', 'tempo_vida_cartao_dias']].head(10))


✔ Features temporais básicas criadas: hora_transacao, dia_semana, data_transacao, fim_de_semana, tempo_vida_cartao_dias


,hora_transacao,dia_semana,fim_de_semana,tempo_vida_cartao_dias
count,30000.0000,30000.0000,30000.0000,30000.0000
mean,11.5055,3.0387,0.2674,1459.3652
std,6.9007,1.9390,0.4426,432.7804
min,0.0000,0.0000,0.0000,694.0000
25%,6.0000,1.0000,0.0000,1083.0000
50%,11.0000,3.0000,0.0000,1461.0000
75%,18.0000,5.0000,1.0000,1835.0000
max,23.0000,6.0000,1.0000,2220.0000


,ts_transacao,dt_emissao_cartao,data_transacao,tempo_vida_cartao_dias
0,2026-01-24 10:54:15,2021-01-02,2026-01-24,1848
1,2026-01-29 10:03:32,2023-02-05,2026-01-29,1089
2,2026-01-03 04:46:30,2022-03-17,2026-01-03,1388
3,2026-01-05 17:53:24,2023-06-30,2026-01-05,920
4,2026-01-16 02:22:42,2024-01-26,2026-01-16,721
5,2026-01-19 16:24:29,2020-02-21,2026-01-19,2159
6,2026-01-24 03:47:09,2022-01-02,2026-01-24,1483
7,2026-01-11 05:17:45,2023-05-21,2026-01-11,966
8,2026-01-12 13:55:35,2022-11-28,2026-01-12,1141
9,2026-01-24 18:17:39,2022-01-19,2026-01-24,1466


<small>***Comentários Letícia:** As features temporais básicas apresentaram distribuição consistente e coerente com o domínio do problema. A variável de tempo de vida do cartão evidenciou uma base predominantemente madura, enquanto as informações de horário, dia da semana e fim de semana capturam padrões de uso potencialmente relevantes para diferenciar comportamentos ao longo do tempo. Essas variáveis formam uma base sólida para análises de sequência e frequência nas etapas seguintes.*</small>

### 6.3 Features de sequência

In [197]:
# Tempo desde a última transação por cartão (em minutos)
df['tempo_desde_ultima_transacao_min'] = (
    df.groupby('id_cartao')['ts_transacao']
      .diff()
      .dt.total_seconds()
      .div(60)
)

# Tempo desde a última transação por cartão (em horas)
df['tempo_desde_ultima_transacao_horas'] = (
    df['tempo_desde_ultima_transacao_min'] / 60
)

# Indicador de uso em intervalo muito curto (menos de 5 minutos)
df['uso_intervalo_curto'] = (
    df['tempo_desde_ultima_transacao_min'] < 5
).astype(int)

checkpoint(
    "Features de sequência criadas: "
    "tempo_desde_ultima_transacao_min, "
    "tempo_desde_ultima_transacao_horas, "
    "uso_intervalo_curto"
)

display(
    df[
        [
            'tempo_desde_ultima_transacao_min',
            'tempo_desde_ultima_transacao_horas',
            'uso_intervalo_curto'
        ]
    ].describe()
)


✔ Features de sequência criadas: tempo_desde_ultima_transacao_min, tempo_desde_ultima_transacao_horas, uso_intervalo_curto


,tempo_desde_ultima_transacao_min,tempo_desde_ultima_transacao_horas,uso_intervalo_curto
count,20517.0000,20517.0000,30000.0000
mean,8729.2180,145.4870,0.0006
std,7674.6236,127.9104,0.0245
min,0.4000,0.0067,0.0000
25%,2771.5833,46.1931,0.0000
50%,6539.1667,108.9861,0.0000
75%,12589.7500,209.8292,0.0000
max,42050.6667,700.8444,1.0000


<small>***Comentários Letícia:** As features de sequência evidenciam que, em média, as transações de um mesmo cartão ocorrem com intervalos relativamente longos, o que é compatível com um padrão regular de uso. O indicador de uso em intervalo muito curto apresentou ocorrência extremamente rara, caracterizando-se como um evento atípico. Esse tipo de variável tende a ser particularmente relevante para a identificação de comportamentos suspeitos pontuais, mesmo que não seja frequente na base.*</small>

### 6.4 Frequência diária

In [198]:
df['qtd_transacoes_dia'] = (
    df.groupby(['id_cartao', 'data_transacao'])['id_transacao']
      .transform('count')
)

checkpoint("Feature criada: qtd_transacoes_dia")

display(df['qtd_transacoes_dia'].describe())

display(
    df[['id_cartao', 'data_transacao', 'id_transacao', 'qtd_transacoes_dia']]
    .head(10)
)

✔ Feature criada: qtd_transacoes_dia


,qtd_transacoes_dia
count,30000.0000
mean,1.1036
std,0.3204
min,1.0000
25%,1.0000
50%,1.0000
75%,1.0000
max,3.0000


,id_cartao,data_transacao,id_transacao,qtd_transacoes_dia
0,10000,2026-01-24,15192,1
1,10000,2026-01-29,24059,1
2,10001,2026-01-03,1599,1
3,10001,2026-01-05,17807,1
4,10001,2026-01-16,21208,1
5,10001,2026-01-19,1696,1
6,10001,2026-01-24,5869,1
7,10002,2026-01-11,4007,1
8,10002,2026-01-12,8432,1
9,10002,2026-01-24,25918,1


<small>***Comentários Letícia:** A frequência diária de uso mostrou que a maior parte dos cartões realiza apenas uma transação por dia, indicando um padrão regular de utilização. Casos com múltiplas transações no mesmo dia são pouco frequentes e representam exceções ao comportamento predominante. Essa variável, embora simples, fornece uma base importante para a construção de indicadores de intensidade de uso nas etapas seguintes.*</small>

### 6.5 Frequência 24h

In [199]:
def _rolling_count_24h(grupo: pd.DataFrame) -> pd.Series:
    # grupo já vem na ordem correta por ts_transacao
    s = (
        grupo.set_index('ts_transacao')['id_transacao']
             .rolling('24h')
             .count()
    )
    # traz de volta o índice original das linhas
    s.index = grupo.index
    return s

df['qtd_transacoes_24h'] = (
    df.groupby('id_cartao', group_keys=False)
      .apply(_rolling_count_24h)
      .round(0)
      .astype('Int64')
)

# Flag de intensidade (threshold conservador)
df['uso_intenso_24h'] = (df['qtd_transacoes_24h'] >= 4).astype(int)

checkpoint("Features criadas: qtd_transacoes_24h, uso_intenso_24h")

display(df[['qtd_transacoes_24h', 'uso_intenso_24h']].describe())

display(
    df[['id_cartao', 'ts_transacao', 'qtd_transacoes_24h', 'uso_intenso_24h']]
    .head(10)
)

✔ Features criadas: qtd_transacoes_24h, uso_intenso_24h


,qtd_transacoes_24h,uso_intenso_24h
count,30000.0000,30000.0000
mean,1.1016,0.0001
std,0.3186,0.0115
min,1.0000,0.0000
25%,1.0000,0.0000
50%,1.0000,0.0000
75%,1.0000,0.0000
max,4.0000,1.0000


,id_cartao,ts_transacao,qtd_transacoes_24h,uso_intenso_24h
0,10000,2026-01-24 10:54:15,1,0
1,10000,2026-01-29 10:03:32,1,0
2,10001,2026-01-03 04:46:30,1,0
3,10001,2026-01-05 17:53:24,1,0
4,10001,2026-01-16 02:22:42,1,0
5,10001,2026-01-19 16:24:29,1,0
6,10001,2026-01-24 03:47:09,1,0
7,10002,2026-01-11 05:17:45,1,0
8,10002,2026-01-12 13:55:35,1,0
9,10002,2026-01-24 18:17:39,1,0


<small>***Comentários Letícia:** A análise da frequência de uso em janelas móveis de 24 horas evidenciou um padrão predominantemente esparso de utilização dos cartões, com a grande maioria apresentando apenas uma transação nesse intervalo. O indicador de uso intenso em 24 horas foi ativado em casos extremamente raros, caracterizando-se como um sinal de exceção. Esse tipo de variável é particularmente útil para capturar comportamentos atípicos pontuais, sem introduzir ruído excessivo no modelo.*</small>

### 6.6 Flags de intensidade

In [200]:
# Garante que o flag existe e está coerente com o threshold definido
# (mantém a regra conservadora: >= 4 transações em 24h)
df['uso_intenso_24h'] = (df['qtd_transacoes_24h'] >= 4).astype(int)

checkpoint("Feature criada/validada: uso_intenso_24h (qtd_transacoes_24h >= 4)")

display(df['uso_intenso_24h'].value_counts(dropna=False).to_frame('contagem'))

display(
    df[['qtd_transacoes_24h', 'uso_intenso_24h']]
    .describe()
)

# Amostra apenas dos casos ativados (se existirem)
display(
    df.loc[df['uso_intenso_24h'] == 1, ['id_cartao', 'ts_transacao', 'qtd_transacoes_24h', 'uso_intenso_24h']]
    .head(10)
)

✔ Feature criada/validada: uso_intenso_24h (qtd_transacoes_24h >= 4)


,contagem
uso_intenso_24h,
0,29996
1,4


,qtd_transacoes_24h,uso_intenso_24h
count,30000.0000,30000.0000
mean,1.1016,0.0001
std,0.3186,0.0115
min,1.0000,0.0000
25%,1.0000,0.0000
50%,1.0000,0.0000
75%,1.0000,0.0000
max,4.0000,1.0000


,id_cartao,ts_transacao,qtd_transacoes_24h,uso_intenso_24h
6680,12205,2026-01-17 15:59:15,4,1
7766,12568,2026-01-18 13:24:46,4,1
12463,14133,2026-01-07 02:42:52,4,1
14347,14780,2026-01-26 09:16:23,4,1


<small>***Comentários Letícia:** Nesta etapa, implementei a flag de intensidade de uso em janela móvel de 24 horas com critério conservador, garantindo que apenas comportamentos realmente atípicos fossem sinalizados. A baixa incidência da feature confirma sua natureza de alerta para eventos extremos, evitando ruído no dataset e preservando seu potencial discriminativo para a modelagem supervisionada.*</small>

### 6.7 Consistência operacional



In [201]:
df['linha_repetida'] = (
    df.groupby('id_cartao')['linha_onibus']
      .transform(lambda s: (s == s.shift(1)).fillna(False).astype(int))
)

df['dispositivo_repetido'] = (
    df.groupby('id_cartao')['id_dispositivo']
      .transform(lambda s: (s == s.shift(1)).fillna(False).astype(int))
)

checkpoint("Features criadas: linha_repetida, dispositivo_repetido")

display(
    df[['linha_repetida', 'dispositivo_repetido']]
    .describe()
)

display(df['linha_repetida'].value_counts(dropna=False).to_frame('contagem_linha_repetida'))
display(df['dispositivo_repetido'].value_counts(dropna=False).to_frame('contagem_dispositivo_repetido'))

display(
    df.loc[
        (df['linha_repetida'] == 1) | (df['dispositivo_repetido'] == 1),
        ['id_cartao', 'ts_transacao', 'linha_onibus', 'id_dispositivo', 'linha_repetida', 'dispositivo_repetido']
    ].head(15)
)


✔ Features criadas: linha_repetida, dispositivo_repetido


,linha_repetida,dispositivo_repetido
count,30000.0000,30000.0000
mean,0.0049,0.0073
std,0.0701,0.0849
min,0.0000,0.0000
25%,0.0000,0.0000
50%,0.0000,0.0000
75%,0.0000,0.0000
max,1.0000,1.0000


,contagem_linha_repetida
linha_repetida,
0,29852
1,148


,contagem_dispositivo_repetido
dispositivo_repetido,
0,29782
1,218


,id_cartao,ts_transacao,linha_onibus,id_dispositivo,linha_repetida,dispositivo_repetido
17,10005,2026-01-15 10:40:16,125,563,1,0
167,10058,2026-01-14 00:05:07,86,502,0,1
203,10065,2026-01-15 12:30:23,133,594,0,1
218,10069,2026-01-21 11:17:36,127,544,0,1
328,10105,2026-01-13 19:45:13,41,561,0,1
409,10132,2026-01-08 04:30:14,99,599,1,0
410,10132,2026-01-09 22:59:53,89,599,0,1
494,10159,2026-01-30 17:04:22,101,550,1,0
516,10166,2026-01-10 08:37:36,77,547,0,1
551,10178,2026-01-15 21:49:22,19,559,0,1


<small>***Comentários Letícia:** Nesta analise, foram construídas features de consistência operacional que comparam cada transação com a imediatamente anterior do mesmo cartão. As variáveis linha_repetida e dispositivo_repetido capturam padrões sequenciais de curto prazo, indicando persistência de linha ou de dispositivo em transações consecutivas. Embora ocorram em baixa frequência, essas situações são relevantes do ponto de vista comportamental, pois podem sinalizar uso atípico quando analisadas em conjunto com métricas de intervalo temporal e intensidade de uso. Essas features enriquecem o modelo ao incorporar informação sequencial que não é capturada por agregações globais.*</small>

### 6.8 Diversidade diária

In [202]:
df['qtd_linhas_distintas_dia'] = (
    df.groupby(['id_cartao', 'data_transacao'])['linha_onibus']
      .transform('nunique')
)

df['qtd_dispositivos_distintos_dia'] = (
    df.groupby(['id_cartao', 'data_transacao'])['id_dispositivo']
      .transform('nunique')
)

checkpoint(
    "Features criadas: qtd_linhas_distintas_dia, qtd_dispositivos_distintos_dia"
)

display(
    df[['qtd_linhas_distintas_dia', 'qtd_dispositivos_distintos_dia']]
    .describe()
)

display(
    df[
        [
            'id_cartao', 'data_transacao', 'linha_onibus', 'id_dispositivo',
            'qtd_linhas_distintas_dia', 'qtd_dispositivos_distintos_dia'
        ]
    ].head(10)
)

✔ Features criadas: qtd_linhas_distintas_dia, qtd_dispositivos_distintos_dia


,qtd_linhas_distintas_dia,qtd_dispositivos_distintos_dia
count,30000.0000,30000.0000
mean,1.1032,1.1026
std,0.3199,0.3188
min,1.0000,1.0000
25%,1.0000,1.0000
50%,1.0000,1.0000
75%,1.0000,1.0000
max,3.0000,3.0000


,id_cartao,data_transacao,linha_onibus,id_dispositivo,qtd_linhas_distintas_dia,qtd_dispositivos_distintos_dia
0,10000,2026-01-24,78,568,1,1
1,10000,2026-01-29,40,524,1,1
2,10001,2026-01-03,53,541,1,1
3,10001,2026-01-05,93,563,1,1
4,10001,2026-01-16,111,572,1,1
5,10001,2026-01-19,136,577,1,1
6,10001,2026-01-24,63,520,1,1
7,10002,2026-01-11,106,568,1,1
8,10002,2026-01-12,136,547,1,1
9,10002,2026-01-24,93,579,1,1


<small>***Comentários Letícia:** Nesta subetapa, foram criadas variáveis que mensuram a diversidade diária de uso do cartão, considerando o número de linhas e dispositivos distintos utilizados em um mesmo dia. Os resultados mostram que o comportamento predominante é o uso único por dia, como já vimos anteriormente, enquanto valores superiores representam exceções comportamentais relevantes. Essas features permitem capturar variações operacionais diárias que, quando combinadas com métricas de intensidade e intervalo temporal, contribuem para a identificação de padrões atípicos associados a possíveis fraudes.*</small>

### 6.9 Features de contexto robustas e simples

In [203]:
# Features de contexto robustas

checkpoint("6.9 Início: criação de features de contexto robustas")

# Idade suspeita
df['idade_suspeita'] = (
    df['idade_usuario'].isna()
    | (df['idade_usuario'] < 0)
    | (df['idade_usuario'] > 110)
).astype(int)

# Feriado binário + flag de mapeamento (apenas se feriado não estiver estritamente 0/1)
feriado_validos = df['feriado'].isin([0, 1]) | df['feriado'].isna()
df['feriado_nao_mapeado'] = (~feriado_validos).astype(int)

df['feriado_bin'] = df['feriado'].where(df['feriado'].isin([0, 1]), 0).fillna(0).astype(int)

# Faixas de temperatura (discretização interpretável)
df['temp_faixa'] = pd.cut(
    df['temp_externa'],
    bins=[-np.inf, 15, 20, 25, 30, np.inf],
    labels=['<=15', '15-20', '20-25', '25-30', '>30'],
    ordered=True
)

checkpoint(
    "Features criadas: idade_suspeita, feriado_bin, feriado_nao_mapeado, temp_faixa"
)

display(
    df[['idade_suspeita', 'feriado_bin', 'feriado_nao_mapeado']]
    .describe()
)

display(df['temp_faixa'].value_counts(dropna=False).to_frame('contagem_temp_faixa'))

display(
    df[['idade_usuario', 'feriado', 'temp_externa', 'idade_suspeita', 'feriado_bin', 'feriado_nao_mapeado', 'temp_faixa']]
    .head(10)
)


✔ 6.9 Início: criação de features de contexto robustas
✔ Features criadas: idade_suspeita, feriado_bin, feriado_nao_mapeado, temp_faixa


,idade_suspeita,feriado_bin,feriado_nao_mapeado
count,30000.0000,30000.0000,30000.0000
mean,0.0019,0.0498,0.0000
std,0.0435,0.2175,0.0000
min,0.0000,0.0000,0.0000
25%,0.0000,0.0000,0.0000
50%,0.0000,0.0000,0.0000
75%,0.0000,0.0000,0.0000
max,1.0000,1.0000,0.0000


,contagem_temp_faixa
temp_faixa,
20-25,10257
25-30,10250
>30,4762
15-20,3998
<=15,733


,idade_usuario,feriado,temp_externa,idade_suspeita,feriado_bin,feriado_nao_mapeado,temp_faixa
0,40,0,28.4229,0,0,0,25-30
1,45,0,30.7558,0,0,0,>30
2,27,0,27.8546,0,0,0,25-30
3,28,0,25.3177,0,0,0,25-30
4,52,0,29.1966,0,0,0,25-30
5,33,0,18.9802,0,0,0,15-20
6,41,0,26.7340,0,0,0,25-30
7,45,0,28.5961,0,0,0,25-30
8,34,0,24.7561,0,0,0,20-25
9,44,0,24.5658,0,0,0,20-25


<small>***Comentários Letícia:** Nesta etapa, foram criadas features de contexto com foco em robustez e qualidade dos dados. A variável idade_suspeita permite sinalizar registros potencialmente inconsistentes sem descartá-los, enquanto feriado_bin e feriado_nao_mapeado garantem a confiabilidade da informação de feriado. A discretização da temperatura em temp_faixa torna o atributo mais interpretável e menos sensível a ruídos. Essas variáveis não têm caráter comportamental direto, mas contribuem para a estabilidade e a capacidade de generalização dos modelos nas etapas posteriores.*</small>

In [204]:
# Features de contexto simples

checkpoint("6.9b Início: criação de features simples de contexto operacional")

# Indicador de sentido da viagem (ida = 1, volta = 0)
df['sentido_ida'] = (df['sentido_viagem'] == 'ida').astype(int)

# Indicador de condição climática potencialmente adversa
df['clima_adverso'] = df['clima_dia'].isin(['chuva', 'nublado']).astype(int)

checkpoint(
    "Features criadas: sentido_ida, clima_adverso"
)

display(
    df[['sentido_ida', 'clima_adverso']]
    .describe()
)

display(
    df[['sentido_viagem', 'clima_dia', 'sentido_ida', 'clima_adverso']]
    .head(10)
)

✔ 6.9b Início: criação de features simples de contexto operacional
✔ Features criadas: sentido_ida, clima_adverso


,sentido_ida,clima_adverso
count,30000.0000,30000.0000
mean,0.5022,0.6666
std,0.5000,0.4714
min,0.0000,0.0000
25%,0.0000,0.0000
50%,1.0000,1.0000
75%,1.0000,1.0000
max,1.0000,1.0000


,sentido_viagem,clima_dia,sentido_ida,clima_adverso
0,volta,chuva,0,1
1,volta,nublado,0,1
2,ida,ensolarado,1,0
3,volta,ensolarado,0,0
4,ida,chuva,1,1
5,ida,nublado,1,1
6,volta,chuva,0,1
7,volta,chuva,0,1
8,ida,ensolarado,1,0
9,volta,chuva,0,1


<small>***Comentários Letícia:** Neste item, foram criadas features simples de contexto operacional, representando o sentido da viagem e condições climáticas adversas. Ambas apresentam boa distribuição e alta interpretabilidade, permitindo capturar aspectos do ambiente da transação que podem influenciar o comportamento de uso. Embora não sejam indicativas diretas de fraude, essas variáveis podem atuar como moderadoras de risco quando combinadas com atributos temporais e de frequência nas etapas de modelagem.*</small>

### 6.10 Faixas de valor

In [205]:
checkpoint("6.10 Início: criação de faixas de valor da transação (robusta a empates)")

# Cria as faixas por quantis de forma robusta a empates
faixas = pd.qcut(df['valor_transacao'], q=4, duplicates='drop')

n_faixas = faixas.cat.categories.size

labels_map = {
    2: ['baixo', 'alto'],
    3: ['baixo', 'medio', 'alto'],
    4: ['baixo', 'medio', 'alto', 'muito_alto']
}

labels = labels_map.get(n_faixas, [f'faixa_{i+1}' for i in range(n_faixas)])

# Aplica labels sem recalcular os quantis
df['valor_transacao_faixa'] = faixas.cat.rename_categories(labels)

checkpoint(f"Feature criada: valor_transacao_faixa ({n_faixas} faixas)")

print("Quantidade de faixas geradas:", n_faixas)
print("Labels aplicadas:", labels)

display(
    df['valor_transacao_faixa']
      .value_counts(dropna=False)
      .sort_index()
)

display(df[['valor_transacao', 'valor_transacao_faixa']].head(10))

✔ 6.10 Início: criação de faixas de valor da transação (robusta a empates)
✔ Feature criada: valor_transacao_faixa (3 faixas)
Quantidade de faixas geradas: 3
Labels aplicadas: ['baixo', 'medio', 'alto']


,count
valor_transacao_faixa,
baixo,22380
medio,5973
alto,1647


,valor_transacao,valor_transacao_faixa
0,4.5000,baixo
1,4.5000,baixo
2,0.0000,baixo
3,4.5000,baixo
4,4.5000,baixo
5,4.5000,baixo
6,4.5000,baixo
7,4.5000,baixo
8,4.5000,baixo
9,4.5000,baixo


<small>***Comentários Letícia:** A criação de faixas para o valor da transação resultou em apenas três grupos efetivos (baixo, médio e alto), apesar da tentativa inicial de segmentação em quatro quantis. Esse comportamento ocorre devido à forte presença de valores repetidos no dataset, especialmente o valor padrão de tarifa, o que faz com que os intervalos se sobreponham e sejam automaticamente colapsados pelo método de discretização. Observei também que a maior parte das transações está concentrada na faixa de menor valor, com uma distribuição residual para valores médios e altos. Essa adaptação automática à distribuição real dos dados evita a criação de faixas artificiais ou vazias, mantendo robustez estatística e preservando a interpretabilidade da variável para análises de negócio e para a etapa de modelagem.*</small>

### 6.11 Agregações por cartão

In [206]:
checkpoint("6.11 Início: agregações por cartão (volume, diversidade, dias ativos e médias)")

# Garantia mínima
if 'data_transacao' not in df.columns:
    df['data_transacao'] = df['ts_transacao'].dt.date

# Colunas que este item vai recriar
cols_611 = [
    'cartao_qtd_transacoes',
    'cartao_dias_ativos',
    'cartao_media_transacoes_por_dia',
    'cartao_qtd_linhas_distintas',
    'cartao_qtd_dispositivos_distintos',
    'cartao_qtd_motoristas_distintos',
    'cartao_valor_transacao_mean',
    'cartao_valor_transacao_std',
    'cartao_pct_integracao',
    'cartao_pct_feriado',
    'cartao_pct_intervalo_curto'
]

# Remove colunas antigas para evitar conflito no merge
cols_existentes = [c for c in cols_611 if c in df.columns]
if cols_existentes:
    df = df.drop(columns=cols_existentes)

# Agregações principais por cartão
agg_cartao = (
    df.groupby('id_cartao')
      .agg(
          cartao_qtd_transacoes=('id_transacao', 'count'),
          cartao_dias_ativos=('data_transacao', 'nunique'),

          cartao_qtd_linhas_distintas=('linha_onibus', 'nunique'),
          cartao_qtd_dispositivos_distintos=('id_dispositivo', 'nunique'),
          cartao_qtd_motoristas_distintos=('id_motorista', 'nunique'),

          cartao_valor_transacao_mean=('valor_transacao', 'mean'),
          cartao_valor_transacao_std=('valor_transacao', 'std'),

          cartao_pct_integracao=('integracao_metro', 'mean'),
          cartao_pct_feriado=('feriado_bin', 'mean'),
          cartao_pct_intervalo_curto=('uso_intervalo_curto', 'mean'),
      )
      .reset_index()
)

# Pós-tratamento leve
agg_cartao['cartao_valor_transacao_std'] = agg_cartao['cartao_valor_transacao_std'].fillna(0)
agg_cartao['cartao_media_transacoes_por_dia'] = (
    agg_cartao['cartao_qtd_transacoes'] / agg_cartao['cartao_dias_ativos'].replace(0, pd.NA)
)

# Merge de volta
df = df.merge(agg_cartao, on='id_cartao', how='left')

# Validação explícita
cols_faltando = [c for c in cols_611 if c not in df.columns]
checkpoint(
    "Features criadas: " + ", ".join(cols_611)
    if not cols_faltando
    else "Atenção: colunas faltando após merge: " + ", ".join(cols_faltando)
)

display(df[cols_611].describe())
display(df[['id_cartao'] + cols_611[:4] + ['cartao_qtd_linhas_distintas']].head(10))

✔ 6.11 Início: agregações por cartão (volume, diversidade, dias ativos e médias)
✔ Features criadas: cartao_qtd_transacoes, cartao_dias_ativos, cartao_media_transacoes_por_dia, cartao_qtd_linhas_distintas, cartao_qtd_dispositivos_distintos, cartao_qtd_motoristas_distintos, cartao_valor_transacao_mean, cartao_valor_transacao_std, cartao_pct_integracao, cartao_pct_feriado, cartao_pct_intervalo_curto


,cartao_qtd_transacoes,cartao_dias_ativos,cartao_media_transacoes_por_dia,cartao_qtd_linhas_distintas,cartao_qtd_dispositivos_distintos,cartao_qtd_motoristas_distintos,cartao_valor_transacao_mean,cartao_valor_transacao_std,cartao_pct_integracao,cartao_pct_feriado,cartao_pct_intervalo_curto
count,30000.0000,30000.0000,30000.0000,30000.0000,30000.0000,30000.0000,30000.0000,30000.0000,30000.0000,30000.0000,30000.0000
mean,4.0103,3.7604,1.0698,3.9582,3.9333,3.9709,12.2525,14.3609,0.4999,0.0498,0.0006
std,1.7405,1.5784,0.1586,1.7081,1.6965,1.7202,46.2070,76.9296,0.2808,0.1221,0.0122
min,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,3.0000,3.0000,1.0000,3.0000,3.0000,3.0000,4.5000,0.0000,0.3333,0.0000,0.0000
50%,4.0000,4.0000,1.0000,4.0000,4.0000,4.0000,5.4000,2.3238,0.5000,0.0000,0.0000
75%,5.0000,5.0000,1.0000,5.0000,5.0000,5.0000,6.7500,3.1820,0.6667,0.0000,0.0000
max,11.0000,10.0000,3.0000,11.0000,11.0000,11.0000,999.9900,707.0997,1.0000,1.0000,0.5000


,id_cartao,cartao_qtd_transacoes,cartao_dias_ativos,cartao_media_transacoes_por_dia,cartao_qtd_linhas_distintas,cartao_qtd_linhas_distintas
0,10000,2,2,1.0000,2,2
1,10000,2,2,1.0000,2,2
2,10001,5,5,1.0000,5,5
3,10001,5,5,1.0000,5,5
4,10001,5,5,1.0000,5,5
5,10001,5,5,1.0000,5,5
6,10001,5,5,1.0000,5,5
7,10002,3,3,1.0000,3,3
8,10002,3,3,1.0000,3,3
9,10002,3,3,1.0000,3,3


<small>***Comentários Letícia:** Neste item, consolidei as agregações por cartão, reunindo volume total de uso, diversidade operacional, dias ativos e médias de utilização diária. Mantive também métricas agregadas de valor e proporções de comportamento, como integração e uso em intervalos curtos, por entender que elas resumem padrões que não aparecem com clareza no nível transacional. O resultado é um conjunto de features estáveis, interpretáveis e coerentes com a distribuição real dos dados, fortalecendo a base para a etapa de modelagem.*</small>

### 6.12 Features comportamentais relativas ao próprio cartão

In [207]:
checkpoint("6.12 Início: features comportamentais relativas ao próprio cartão")

df = df.sort_values(['id_cartao', 'ts_transacao']).reset_index(drop=True)

# 6.12.1 Período do dia (categórico interpretável)
hora = df['hora_transacao'] if 'hora_transacao' in df.columns else df['ts_transacao'].dt.hour

df['periodo_dia'] = pd.cut(
    hora,
    bins=[-0.1, 5, 11, 17, 23],
    labels=['madrugada', 'manha', 'tarde', 'noite']
).astype('category')

# 6.12.2 Valor atual vs padrão do próprio cartão (média e desvio do cartão)
cartao_stats_valor = (
    df.groupby('id_cartao')['valor_transacao']
      .agg(cartao_valor_mean='mean', cartao_valor_std='std')
)

cartao_stats_valor['cartao_valor_std'] = cartao_stats_valor['cartao_valor_std'].fillna(0)

df = df.join(cartao_stats_valor, on='id_cartao')

eps = 1e-9
df['valor_vs_media_cartao'] = (df['valor_transacao'] - df['cartao_valor_mean']).abs()
df['valor_zscore_cartao'] = (df['valor_transacao'] - df['cartao_valor_mean']) / (df['cartao_valor_std'] + eps)

# Flags conservadoras de "fora do padrão" do próprio cartão
df['valor_outlier_cartao'] = (
    (df['cartao_valor_std'] > 0) & (df['valor_zscore_cartao'].abs() >= 3)
).astype(int)

# 6.12.3 Uso acima do padrão diário do cartão (baseado em transações/dia do cartão)
if 'qtd_transacoes_dia' in df.columns and 'cartao_media_transacoes_por_dia' in df.columns:
    df['uso_acima_media_dia_cartao'] = (
        df['qtd_transacoes_dia'] > df['cartao_media_transacoes_por_dia']
    ).astype(int)
else:
    df['uso_acima_media_dia_cartao'] = 0

checkpoint(
    "Features criadas: periodo_dia, valor_vs_media_cartao, valor_zscore_cartao, "
    "valor_outlier_cartao, uso_acima_media_dia_cartao"
)

display(
    df[
        [
            'valor_vs_media_cartao',
            'valor_zscore_cartao',
            'valor_outlier_cartao',
            'uso_acima_media_dia_cartao'
        ]
    ].describe()
)

display(df[['hora_transacao' if 'hora_transacao' in df.columns else 'ts_transacao', 'periodo_dia']].head(10))

display(
    df[
        [
            'id_cartao',
            'valor_transacao',
            'cartao_valor_mean',
            'cartao_valor_std',
            'valor_vs_media_cartao',
            'valor_zscore_cartao',
            'valor_outlier_cartao',
            'qtd_transacoes_dia' if 'qtd_transacoes_dia' in df.columns else 'id_transacao',
            'cartao_media_transacoes_por_dia' if 'cartao_media_transacoes_por_dia' in df.columns else 'id_transacao',
            'uso_acima_media_dia_cartao'
        ]
    ].head(10)
)

✔ 6.12 Início: features comportamentais relativas ao próprio cartão
✔ Features criadas: periodo_dia, valor_vs_media_cartao, valor_zscore_cartao, valor_outlier_cartao, uso_acima_media_dia_cartao


,valor_vs_media_cartao,valor_zscore_cartao,valor_outlier_cartao,uso_acima_media_dia_cartao
count,30000.0000,30000.0000,30000.0000,30000.0000
mean,10.4074,-0.0000,0.0000,0.0925
std,65.6680,0.7169,0.0000,0.2897
min,0.0000,-2.6667,0.0000,0.0000
25%,0.0000,-0.5000,0.0000,0.0000
50%,1.1250,0.0000,0.0000,0.0000
75%,2.7000,0.4082,0.0000,0.0000
max,871.0538,2.8460,0.0000,1.0000


,hora_transacao,periodo_dia
0,10,manha
1,10,manha
2,4,madrugada
3,17,tarde
4,2,madrugada
5,16,tarde
6,3,madrugada
7,5,madrugada
8,13,tarde
9,18,noite


,id_cartao,valor_transacao,cartao_valor_mean,cartao_valor_std,valor_vs_media_cartao,valor_zscore_cartao,valor_outlier_cartao,qtd_transacoes_dia,cartao_media_transacoes_por_dia,uso_acima_media_dia_cartao
0,10000,4.5000,4.5000,0.0000,0.0000,0.0000,0,1,1.0000,0
1,10000,4.5000,4.5000,0.0000,0.0000,0.0000,0,1,1.0000,0
2,10001,0.0000,3.6000,2.0125,3.6000,-1.7889,0,1,1.0000,0
3,10001,4.5000,3.6000,2.0125,0.9000,0.4472,0,1,1.0000,0
4,10001,4.5000,3.6000,2.0125,0.9000,0.4472,0,1,1.0000,0
5,10001,4.5000,3.6000,2.0125,0.9000,0.4472,0,1,1.0000,0
6,10001,4.5000,3.6000,2.0125,0.9000,0.4472,0,1,1.0000,0
7,10002,4.5000,4.5000,0.0000,0.0000,0.0000,0,1,1.0000,0
8,10002,4.5000,4.5000,0.0000,0.0000,0.0000,0,1,1.0000,0
9,10002,4.5000,4.5000,0.0000,0.0000,0.0000,0,1,1.0000,0


<small>***Comentários Letícia:** Decidi incluir esta etapa adicional para criar features que comparam cada transação com o próprio histórico do cartão, indo além de métricas globais ou puramente agregadas. A proposta aqui foi capturar desvios comportamentais individuais, que muitas vezes não aparecem quando observo apenas médias gerais do sistema.*</small>

<small>*Criei a variável periodo_dia para transformar o horário exato da transação em categorias mais interpretáveis, permitindo representar padrões de uso associados a momentos específicos do dia, como madrugada, manhã, tarde e noite. Essa discretização ajuda a organizar o comportamento temporal sem perder informação relevante.*</small>

<small>*A feature valor_vs_media_cartao foi construída para medir o quanto o valor da transação se afasta da média histórica do próprio cartão, permitindo identificar situações em que o uso foge do padrão habitual do titular. Em complemento, a valor_zscore_cartao padroniza esse desvio considerando a variabilidade do cartão, o que torna comparáveis cartões com perfis de gasto distintos.*</small>

<small>*Incluí também a valor_outlier_cartao, baseada em um critério estatístico objetivo, para marcar transações com valores extremamente discrepantes em relação ao histórico do cartão. Mesmo que, neste dataset, poucos casos tenham atendido ao critério, optei por manter a feature por coerência metodológica.*</small>

<small>*Por fim, criei a uso_acima_media_dia_cartao para indicar situações em que o volume diário de uso supera a média histórica do cartão, buscando capturar aumentos pontuais de intensidade que não dependem apenas de janelas fixas de tempo.*</small>

<small>*Essa etapa foi pensada exclusivamente para enriquecer a representação comportamental do cartão, deixando a avaliação prática da relevância dessas variáveis para a etapa seguinte de consolidação e análise final do conjunto de features.*</small>

### 6.13 Checkpoint geral das features

In [208]:
# Lista explícita de todas as features criadas na Etapa 6
features_etapa_6 = [
    # Temporais
    'hora_transacao',
    'dia_semana',
    'data_transacao',
    'fim_de_semana',
    'tempo_vida_cartao_dias',

    # Sequência
    'tempo_desde_ultima_transacao_min',
    'tempo_desde_ultima_transacao_horas',
    'uso_intervalo_curto',

    # Frequência
    'qtd_transacoes_dia',
    'qtd_transacoes_24h',
    'uso_intenso_24h',

    # Consistência operacional
    'linha_repetida',
    'dispositivo_repetido',

    # Diversidade diária
    'qtd_linhas_distintas_dia',
    'qtd_dispositivos_distintos_dia',

    # Contexto robusto
    'idade_suspeita',
    'feriado_bin',
    'feriado_nao_mapeado',
    'temp_faixa',

    # Contexto simples
    'sentido_ida',
    'clima_adverso',

    # Faixas de valor
    'valor_transacao_faixa',

    # Agregações por cartão
    'cartao_qtd_transacoes',
    'cartao_dias_ativos',
    'cartao_media_transacoes_por_dia',
    'cartao_qtd_linhas_distintas',
    'cartao_qtd_dispositivos_distintos',
    'cartao_qtd_motoristas_distintos',
    'cartao_valor_transacao_mean',
    'cartao_valor_transacao_std',
    'cartao_pct_integracao',
    'cartao_pct_feriado',
    'cartao_pct_intervalo_curto',

    # Comportamentais do próprio cartão
    'periodo_dia',
    'valor_vs_media_cartao',
    'valor_zscore_cartao',
    'valor_outlier_cartao',
    'uso_acima_media_dia_cartao'
]

# Verificação de existência
features_ausentes = [f for f in features_etapa_6 if f not in df.columns]

print(f"Total de features previstas na Etapa 6: {len(features_etapa_6)}")
print("Features ausentes:", features_ausentes)

# Checagem de valores ausentes
print("\nPercentual de valores ausentes nas features da Etapa 6:")
display(
    df[features_etapa_6]
    .isna()
    .mean()
    .mul(100)
    .round(2)
    .sort_values(ascending=False)
)

# Amostra final
print("\nAmostra das features criadas:")
display(df[features_etapa_6].head(10))

checkpoint("6.13 Concluído: todas as features da Etapa 6 consolidadas")


Total de features previstas na Etapa 6: 38
Features ausentes: []

Percentual de valores ausentes nas features da Etapa 6:


,0
tempo_desde_ultima_transacao_min,31.6100
tempo_desde_ultima_transacao_horas,31.6100
dia_semana,0.0000
data_transacao,0.0000
fim_de_semana,0.0000
hora_transacao,0.0000
tempo_vida_cartao_dias,0.0000
uso_intervalo_curto,0.0000
qtd_transacoes_dia,0.0000
qtd_transacoes_24h,0.0000



Amostra das features criadas:


,hora_transacao,dia_semana,data_transacao,fim_de_semana,tempo_vida_cartao_dias,tempo_desde_ultima_transacao_min,tempo_desde_ultima_transacao_horas,uso_intervalo_curto,qtd_transacoes_dia,qtd_transacoes_24h,uso_intenso_24h,linha_repetida,dispositivo_repetido,qtd_linhas_distintas_dia,qtd_dispositivos_distintos_dia,idade_suspeita,feriado_bin,feriado_nao_mapeado,temp_faixa,sentido_ida,clima_adverso,valor_transacao_faixa,cartao_qtd_transacoes,cartao_dias_ativos,cartao_media_transacoes_por_dia,cartao_qtd_linhas_distintas,cartao_qtd_dispositivos_distintos,cartao_qtd_motoristas_distintos,cartao_valor_transacao_mean,cartao_valor_transacao_std,cartao_pct_integracao,cartao_pct_feriado,cartao_pct_intervalo_curto,periodo_dia,valor_vs_media_cartao,valor_zscore_cartao,valor_outlier_cartao,uso_acima_media_dia_cartao
0,10,5,2026-01-24,1,1848,NaN,NaN,0,1,1,0,0,0,1,1,0,0,0,25-30,0,1,baixo,2,2,1.0000,2,2,2,4.5000,0.0000,0.0000,0.0000,0.0000,manha,0.0000,0.0000,0,0
1,10,3,2026-01-29,0,1089,7149.2833,119.1547,0,1,1,0,0,0,1,1,0,0,0,>30,0,1,baixo,2,2,1.0000,2,2,2,4.5000,0.0000,0.0000,0.0000,0.0000,manha,0.0000,0.0000,0,0
2,4,5,2026-01-03,1,1388,NaN,NaN,0,1,1,0,0,0,1,1,0,0,0,25-30,1,0,baixo,5,5,1.0000,5,5,5,3.6000,2.0125,0.4000,0.0000,0.0000,madrugada,3.6000,-1.7889,0,0
3,17,0,2026-01-05,0,920,3666.9000,61.1150,0,1,1,0,0,0,1,1,0,0,0,25-30,0,0,baixo,5,5,1.0000,5,5,5,3.6000,2.0125,0.4000,0.0000,0.0000,tarde,0.9000,0.4472,0,0
4,2,4,2026-01-16,0,721,14909.3000,248.4883,0,1,1,0,0,0,1,1,0,0,0,25-30,1,1,baixo,5,5,1.0000,5,5,5,3.6000,2.0125,0.4000,0.0000,0.0000,madrugada,0.9000,0.4472,0,0
5,16,0,2026-01-19,0,2159,5161.7833,86.0297,0,1,1,0,0,0,1,1,0,0,0,15-20,1,1,baixo,5,5,1.0000,5,5,5,3.6000,2.0125,0.4000,0.0000,0.0000,tarde,0.9000,0.4472,0,0
6,3,5,2026-01-24,1,1483,6442.6667,107.3778,0,1,1,0,0,0,1,1,0,0,0,25-30,0,1,baixo,5,5,1.0000,5,5,5,3.6000,2.0125,0.4000,0.0000,0.0000,madrugada,0.9000,0.4472,0,0
7,5,6,2026-01-11,1,966,NaN,NaN,0,1,1,0,0,0,1,1,0,0,0,25-30,0,1,baixo,3,3,1.0000,3,3,3,4.5000,0.0000,0.3333,0.0000,0.0000,madrugada,0.0000,0.0000,0,0
8,13,0,2026-01-12,0,1141,1957.8333,32.6306,0,1,1,0,0,0,1,1,0,0,0,20-25,1,0,baixo,3,3,1.0000,3,3,3,4.5000,0.0000,0.3333,0.0000,0.0000,tarde,0.0000,0.0000,0,0
9,18,5,2026-01-24,1,1466,17542.0667,292.3678,0,1,1,0,0,0,1,1,0,0,0,20-25,0,1,baixo,3,3,1.0000,3,3,3,4.5000,0.0000,0.3333,0.0000,0.0000,noite,0.0000,0.0000,0,0


✔ 6.13 Concluído: todas as features da Etapa 6 consolidadas


#### Conclusões da Engenharia de Features

A Etapa 6 consolidou a criação de 38 features novas, todas validadas quanto à consistência, ausência de vazamento e interpretabilidade. O conjunto resultante está preparado para a seleção final de colunas e para a comparação entre diferentes abordagens de modelagem na próxima etapa do projeto.

#### 1. Features temporais e de sequência
**Features criadas:**  
- `hora_transacao`, `dia_semana`, `data_transacao`, `fim_de_semana`  
- `tempo_vida_cartao_dias`  
- `tempo_desde_ultima_transacao_min`, `tempo_desde_ultima_transacao_horas`  
- `uso_intervalo_curto`

Essas variáveis descrevem quando a transação ocorreu e como ela se encaixa na sequência histórica do cartão. O tempo desde a última transação e a flag de intervalo curto permitem identificar padrões de uso concentrados no tempo.

**Modelos que tendem a se beneficiar:**  
- Modelos baseados em árvores  
- Métodos de ensemble

#### 2. Frequência e intensidade de uso
**Features criadas:**  
- `qtd_transacoes_dia`  
- `qtd_transacoes_24h`  
- `uso_intenso_24h`

Essas features quantificam a intensidade de uso do cartão em janelas temporais distintas, permitindo diferenciar comportamentos regulares de padrões intensivos.

**Modelos que tendem a se beneficiar:**  
- Árvores de decisão e boosting  
- Modelos lineares como variáveis auxiliares

#### 3. Consistência operacional
**Features criadas:**  
- `linha_repetida`  
- `dispositivo_repetido`

Essas variáveis indicam se a transação ocorreu na mesma linha ou no mesmo dispositivo da transação anterior do cartão, capturando repetições operacionais imediatas.

**Modelos que tendem a se beneficiar:**  
- Árvores  
- Modelos capazes de capturar interações não lineares

#### 4. Diversidade diária
**Features criadas:**  
- `qtd_linhas_distintas_dia`  
- `qtd_dispositivos_distintos_dia`

Essas features medem a diversidade operacional do cartão dentro de um mesmo dia, refletindo deslocamentos e variações concentradas em curto intervalo de tempo.

**Modelos que tendem a se beneficiar:**  
- Árvores  
- Modelos com foco em padrões locais

#### 5. Contexto robusto e simples
**Features robustas:**  
- `idade_suspeita`, `feriado_bin`, `feriado_nao_mapeado`, `temp_faixa`  

**Features simples:**  
- `sentido_ida`, `clima_adverso`

Essas variáveis incorporam contexto externo e indicadores de qualidade dos dados, ajudando a diferenciar comportamentos sob condições operacionais distintas.

**Modelos que tendem a se beneficiar:**  
- Modelos lineares  
- Árvores, como variáveis auxiliares

#### 6. Agregações por cartão
**Features criadas:**  
- Volume e atividade: `cartao_qtd_transacoes`, `cartao_dias_ativos`, `cartao_media_transacoes_por_dia`  
- Diversidade: `cartao_qtd_linhas_distintas`, `cartao_qtd_dispositivos_distintos`, `cartao_qtd_motoristas_distintos`  
- Estatísticas: `cartao_valor_transacao_mean`, `cartao_valor_transacao_std`  
- Proporções: `cartao_pct_integracao`, `cartao_pct_feriado`, `cartao_pct_intervalo_curto`

Essas agregações sintetizam o perfil histórico do cartão, permitindo avaliar cada transação em relação ao comportamento médio do titular.

**Modelos que tendem a se beneficiar:**  
- Todos os modelos avaliados  
- Especialmente modelos lineares e boosting

#### 7. Features comportamentais relativas ao próprio cartão
**Features criadas:**  
- `periodo_dia`  
- `valor_vs_media_cartao`, `valor_zscore_cartao`, `valor_outlier_cartao`  
- `uso_acima_media_dia_cartao`

Essas variáveis comparam cada transação com o histórico do próprio cartão, capturando desvios individuais sem depender de padrões globais.

**Modelos que tendem a se beneficiar:**  
- Árvores e boosting  
- Modelos lineares, pela padronização implícita


## 7. Seleção final de colunas e preparação do dataset processado

### 7.1 Definição de colunas de rastreio, target e features

In [209]:
# Colunas de rastreio (mantidas para auditoria/debug, não para o modelo)
cols_rastreio = ['id_transacao', 'id_cartao', 'ts_transacao']

# Variável alvo
col_target = 'target_fraude'

# Features
features_etapa_6 = [
    'hora_transacao',
    'dia_semana',
    'data_transacao',
    'fim_de_semana',
    'tempo_vida_cartao_dias',
    'tempo_desde_ultima_transacao_min',
    'tempo_desde_ultima_transacao_horas',
    'uso_intervalo_curto',
    'qtd_transacoes_dia',
    'qtd_transacoes_24h',
    'uso_intenso_24h',
    'linha_repetida',
    'dispositivo_repetido',
    'qtd_linhas_distintas_dia',
    'qtd_dispositivos_distintos_dia',
    'idade_suspeita',
    'feriado_bin',
    'feriado_nao_mapeado',
    'temp_faixa',
    'sentido_ida',
    'clima_adverso',
    'valor_transacao_faixa',
    'cartao_qtd_transacoes',
    'cartao_dias_ativos',
    'cartao_media_transacoes_por_dia',
    'cartao_qtd_linhas_distintas',
    'cartao_qtd_dispositivos_distintos',
    'cartao_qtd_motoristas_distintos',
    'cartao_valor_transacao_mean',
    'cartao_valor_transacao_std',
    'cartao_pct_integracao',
    'cartao_pct_feriado',
    'cartao_pct_intervalo_curto',
    'periodo_dia',
    'valor_vs_media_cartao',
    'valor_zscore_cartao',
    'valor_outlier_cartao',
    'uso_acima_media_dia_cartao'
]

# Validações de existência
obrigatorias = cols_rastreio + [col_target]
faltando_obrigatorias = [c for c in obrigatorias if c not in df.columns]
faltando_features = [f for f in features_etapa_6 if f not in df.columns]

print("Colunas de rastreio:", cols_rastreio)
print("Target:", col_target)
print("Total de features previstas (Etapa 6):", len(features_etapa_6))

print("\nChecagem de existência:")
print("Faltando obrigatórias:", faltando_obrigatorias)
print("Faltando features:", faltando_features)

# Checagem de interseção indevida
intersecao_rastreio = sorted(set(cols_rastreio).intersection(features_etapa_6))
intersecao_target = sorted(set([col_target]).intersection(features_etapa_6))

print("\nChecagem de contrato (sem vazamentos estruturais):")
print("Interseção rastreio ∩ features:", intersecao_rastreio)
print("Interseção target ∩ features:", intersecao_target)

# Lista final de features válidas (somente as que existem no df)
features_validas = [f for f in features_etapa_6 if f in df.columns]

checkpoint(
    f"7.1 Concluído: contrato definido | rastreio={len(cols_rastreio)} "
    f"| target=1 | features_validas={len(features_validas)}"
)

display(pd.DataFrame({
    'grupo': (['rastreio'] * len(cols_rastreio)) + (['target']) + (['feature'] * len(features_validas)),
    'coluna': cols_rastreio + [col_target] + features_validas
}))

Colunas de rastreio: ['id_transacao', 'id_cartao', 'ts_transacao']
Target: target_fraude
Total de features previstas (Etapa 6): 38

Checagem de existência:
Faltando obrigatórias: []
Faltando features: []

Checagem de contrato (sem vazamentos estruturais):
Interseção rastreio ∩ features: []
Interseção target ∩ features: []
✔ 7.1 Concluído: contrato definido | rastreio=3 | target=1 | features_validas=38


,grupo,coluna
0,rastreio,id_transacao
1,rastreio,id_cartao
2,rastreio,ts_transacao
3,target,target_fraude
4,feature,hora_transacao
5,feature,dia_semana
6,feature,data_transacao
7,feature,fim_de_semana
8,feature,tempo_vida_cartao_dias
9,feature,tempo_desde_ultima_transacao_min


<small>***Comentários Letícia:** Eu defini um detalhamento claro para o dataset final, separando colunas de rastreio, target e as 38 features da etapa 6. Eu validei que nenhuma coluna obrigatória ficou faltando e confirmei que não existe sobreposição entre rastreio, target e features, evitando vazamentos estruturais e entradas indevidas no conjunto de variáveis do modelo. Eu também gerei um manifesto com o agrupamento de cada coluna, deixando o pipeline mais auditável e fácil de manter nas próximas etapas.*</small>

### 7.2 Montagem do dataset final

In [210]:
# Colunas de rastreio, target e features
cols_rastreio = ['id_transacao', 'id_cartao', 'ts_transacao']
target = ['target_fraude']

features = [
    'hora_transacao',
    'dia_semana',
    'data_transacao',
    'fim_de_semana',
    'tempo_vida_cartao_dias',
    'tempo_desde_ultima_transacao_min',
    'tempo_desde_ultima_transacao_horas',
    'uso_intervalo_curto',
    'qtd_transacoes_dia',
    'qtd_transacoes_24h',
    'uso_intenso_24h',
    'linha_repetida',
    'dispositivo_repetido',
    'qtd_linhas_distintas_dia',
    'qtd_dispositivos_distintos_dia',
    'idade_suspeita',
    'feriado_bin',
    'feriado_nao_mapeado',
    'temp_faixa',
    'sentido_ida',
    'clima_adverso',
    'valor_transacao_faixa',
    'cartao_qtd_transacoes',
    'cartao_dias_ativos',
    'cartao_media_transacoes_por_dia',
    'cartao_qtd_linhas_distintas',
    'cartao_qtd_dispositivos_distintos',
    'cartao_qtd_motoristas_distintos',
    'cartao_valor_transacao_mean',
    'cartao_valor_transacao_std',
    'cartao_pct_integracao',
    'cartao_pct_feriado',
    'cartao_pct_intervalo_curto',
    'periodo_dia',
    'valor_vs_media_cartao',
    'valor_zscore_cartao',
    'valor_outlier_cartao',
    'uso_acima_media_dia_cartao'
]

final_cols = cols_rastreio + target + features

# Montagem do dataset final
df_final = df[final_cols].copy()

checkpoint(
    f"7.2 Concluído: df_final montado | shape={df_final.shape} "
    f"| rastreio={len(cols_rastreio)} | target={len(target)} | features={len(features)}"
)

# Visualizações de validação rápida
display(df_final.head())
display(df_final.tail())

✔ 7.2 Concluído: df_final montado | shape=(30000, 42) | rastreio=3 | target=1 | features=38


,id_transacao,id_cartao,ts_transacao,target_fraude,hora_transacao,dia_semana,data_transacao,fim_de_semana,tempo_vida_cartao_dias,tempo_desde_ultima_transacao_min,tempo_desde_ultima_transacao_horas,uso_intervalo_curto,qtd_transacoes_dia,qtd_transacoes_24h,uso_intenso_24h,linha_repetida,dispositivo_repetido,qtd_linhas_distintas_dia,qtd_dispositivos_distintos_dia,idade_suspeita,feriado_bin,feriado_nao_mapeado,temp_faixa,sentido_ida,clima_adverso,valor_transacao_faixa,cartao_qtd_transacoes,cartao_dias_ativos,cartao_media_transacoes_por_dia,cartao_qtd_linhas_distintas,cartao_qtd_dispositivos_distintos,cartao_qtd_motoristas_distintos,cartao_valor_transacao_mean,cartao_valor_transacao_std,cartao_pct_integracao,cartao_pct_feriado,cartao_pct_intervalo_curto,periodo_dia,valor_vs_media_cartao,valor_zscore_cartao,valor_outlier_cartao,uso_acima_media_dia_cartao
0,15192,10000,2026-01-24 10:54:15,1,10,5,2026-01-24,1,1848,NaN,NaN,0,1,1,0,0,0,1,1,0,0,0,25-30,0,1,baixo,2,2,1.0000,2,2,2,4.5000,0.0000,0.0000,0.0000,0.0000,manha,0.0000,0.0000,0,0
1,24059,10000,2026-01-29 10:03:32,0,10,3,2026-01-29,0,1089,7149.2833,119.1547,0,1,1,0,0,0,1,1,0,0,0,>30,0,1,baixo,2,2,1.0000,2,2,2,4.5000,0.0000,0.0000,0.0000,0.0000,manha,0.0000,0.0000,0,0
2,1599,10001,2026-01-03 04:46:30,0,4,5,2026-01-03,1,1388,NaN,NaN,0,1,1,0,0,0,1,1,0,0,0,25-30,1,0,baixo,5,5,1.0000,5,5,5,3.6000,2.0125,0.4000,0.0000,0.0000,madrugada,3.6000,-1.7889,0,0
3,17807,10001,2026-01-05 17:53:24,0,17,0,2026-01-05,0,920,3666.9000,61.1150,0,1,1,0,0,0,1,1,0,0,0,25-30,0,0,baixo,5,5,1.0000,5,5,5,3.6000,2.0125,0.4000,0.0000,0.0000,tarde,0.9000,0.4472,0,0
4,21208,10001,2026-01-16 02:22:42,0,2,4,2026-01-16,0,721,14909.3000,248.4883,0,1,1,0,0,0,1,1,0,0,0,25-30,1,1,baixo,5,5,1.0000,5,5,5,3.6000,2.0125,0.4000,0.0000,0.0000,madrugada,0.9000,0.4472,0,0


,id_transacao,id_cartao,ts_transacao,target_fraude,hora_transacao,dia_semana,data_transacao,fim_de_semana,tempo_vida_cartao_dias,tempo_desde_ultima_transacao_min,tempo_desde_ultima_transacao_horas,uso_intervalo_curto,qtd_transacoes_dia,qtd_transacoes_24h,uso_intenso_24h,linha_repetida,dispositivo_repetido,qtd_linhas_distintas_dia,qtd_dispositivos_distintos_dia,idade_suspeita,feriado_bin,feriado_nao_mapeado,temp_faixa,sentido_ida,clima_adverso,valor_transacao_faixa,cartao_qtd_transacoes,cartao_dias_ativos,cartao_media_transacoes_por_dia,cartao_qtd_linhas_distintas,cartao_qtd_dispositivos_distintos,cartao_qtd_motoristas_distintos,cartao_valor_transacao_mean,cartao_valor_transacao_std,cartao_pct_integracao,cartao_pct_feriado,cartao_pct_intervalo_curto,periodo_dia,valor_vs_media_cartao,valor_zscore_cartao,valor_outlier_cartao,uso_acima_media_dia_cartao
29995,8290,19998,2026-01-23 17:29:23,0,17,4,2026-01-23,0,1831,23432.7833,390.5464,0,1,1,0,0,0,1,1,0,0,0,20-25,1,1,medio,5,5,1.0000,5,5,5,7.2000,4.0249,0.6000,0.2000,0.0000,tarde,1.8000,0.4472,0,0
29996,13216,19998,2026-01-24 06:12:49,0,6,5,2026-01-24,1,997,763.4333,12.7239,0,1,2,0,0,0,1,1,0,1,0,25-30,0,1,baixo,5,5,1.0000,5,5,5,7.2000,4.0249,0.6000,0.2000,0.0000,manha,2.7000,-0.6708,0,0
29997,23457,19998,2026-01-30 21:13:58,1,21,4,2026-01-30,0,2091,9541.1500,159.0192,0,1,1,0,0,0,1,1,0,0,0,25-30,0,0,alto,5,5,1.0000,5,5,5,7.2000,4.0249,0.6000,0.2000,0.0000,noite,6.3000,1.5652,0,0
29998,21422,19999,2026-01-08 11:05:59,0,11,3,2026-01-08,0,1039,NaN,NaN,0,1,1,0,0,0,1,1,0,0,0,25-30,1,0,medio,2,2,1.0000,2,2,2,6.7500,3.1820,0.5000,0.0000,0.0000,manha,2.2500,0.7071,0,0
29999,19013,19999,2026-01-23 20:38:49,0,20,4,2026-01-23,0,2061,22172.8333,369.5472,0,1,1,0,0,0,1,1,0,0,0,>30,0,1,baixo,2,2,1.0000,2,2,2,6.7500,3.1820,0.5000,0.0000,0.0000,noite,2.2500,-0.7071,0,0


<small>***Comentários Letícia:** No item 7.2, consolidei o dataset final reunindo as colunas de rastreio, a variável alvo e todas as features criadas na etapa de engenharia. Decidi manter uma seleção explícita e controlada das colunas, sem aplicar novos tratamentos ou transformações neste momento, para preservar a integridade das variáveis e facilitar a rastreabilidade das observações. O resultado é uma base transacional organizada e consistente, preparada para as validações finais e para a etapa de modelagem.*</small>

### 7.3 Validação de dtypes finais e viabilidade de encoding

In [211]:
# Visão geral dos tipos de dados
dtypes_df = (
    df_final.dtypes
    .reset_index()
    .rename(columns={'index': 'coluna', 0: 'dtype'})
)

display(dtypes_df)

# Identificação de colunas categóricas
cols_categoricas = dtypes_df[
    dtypes_df['dtype'].isin(['object', 'category'])
]['coluna'].tolist()

print("Colunas categóricas identificadas:")
print(cols_categoricas)

# Cardinalidade das categóricas
cardinalidade = (
    df_final[cols_categoricas]
    .nunique()
    .sort_values(ascending=False)
)

display(cardinalidade)

checkpoint(
    "7.3 Concluído: dtypes validados e categóricas mapeadas para decisão de encoding"
)

,coluna,dtype
0,id_transacao,Int64
1,id_cartao,Int64
2,ts_transacao,datetime64[ns]
3,target_fraude,Int64
4,hora_transacao,int32
5,dia_semana,int32
6,data_transacao,object
7,fim_de_semana,int64
8,tempo_vida_cartao_dias,int64
9,tempo_desde_ultima_transacao_min,float64


Colunas categóricas identificadas:
['valor_transacao_faixa', 'periodo_dia']


,0
periodo_dia,4
valor_transacao_faixa,3


✔ 7.3 Concluído: dtypes validados e categóricas mapeadas para decisão de encoding


<small>***Comentários Letícia:** Aqui validei os tipos finais das variáveis e identifiquei explicitamente quais colunas exigem encoding antes da modelagem. Observei que o dataset final ficou majoritariamente numérico, com apenas duas variáveis categóricas de baixa cardinalidade, o que simplifica a etapa de pré-processamento e reduz riscos de inconsistência nos modelos. Essa checagem me deu segurança de que as features estão tecnicamente adequadas para serem usadas em diferentes algoritmos, sem ajustes improvisados na fase de treino.*</small>

### 7.4 Auditoria final de nulos

In [212]:
# Percentual de valores ausentes por coluna
nulos_pct = (
    df_final
    .isna()
    .mean()
    .mul(100)
    .round(2)
    .sort_values(ascending=False)
)

# Colunas com pelo menos 1 valor ausente
nulos_presentes = nulos_pct[nulos_pct > 0]

print("Colunas com valores ausentes:")
display(nulos_presentes)

print(f"Total de colunas com nulos: {nulos_presentes.shape[0]}")
print(f"Percentual máximo de nulos em uma coluna: {nulos_presentes.max():.2f}%")

# Quantidade de linhas afetadas por pelo menos um nulo
linhas_com_nulos = df_final.isna().any(axis=1).mean() * 100
print(f"Percentual de registros com pelo menos um valor ausente: {linhas_com_nulos:.2f}%")

checkpoint("7.4 Concluído: auditoria final de nulos realizada")


Colunas com valores ausentes:


,0
tempo_desde_ultima_transacao_min,31.6100
tempo_desde_ultima_transacao_horas,31.6100


Total de colunas com nulos: 2
Percentual máximo de nulos em uma coluna: 31.61%
Percentual de registros com pelo menos um valor ausente: 31.61%
✔ 7.4 Concluído: auditoria final de nulos realizada


<small>***Comentários Letícia:** No fechamento da auditoria de valores ausentes, verifiquei que apenas as variáveis de intervalo temporal entre transações apresentaram nulos, exatamente nos registros que representam a primeira utilização de cada cartão. Esse comportamento é esperado e coerente com a lógica de construção das features de sequência. Não identifiquei ausências indevidas em variáveis críticas, agregadas ou no target, o que confirma a consistência do dataset final antes da etapa de modelagem.*</small>

### 7.5 Checkpoint do dataset final

In [213]:
print(f"Shape final do df_final: {df_final.shape}")

# Contagem por tipo lógico
qtd_rastreio = len(cols_rastreio)
qtd_target = 1
qtd_features = len([c for c in df_final.columns if c not in cols_rastreio + target])

print(f"Colunas de rastreio: {qtd_rastreio}")
print(f"Coluna target: {qtd_target}")
print(f"Total de features: {qtd_features}")

# Lista ordenada de colunas
print("\nLista final de colunas (ordem do dataset):")
for c in df_final.columns:
    print("-", c)

checkpoint("7.5 Concluído: dataset final validado e pronto para exportação")


Shape final do df_final: (30000, 42)
Colunas de rastreio: 3
Coluna target: 1
Total de features: 38

Lista final de colunas (ordem do dataset):
- id_transacao
- id_cartao
- ts_transacao
- target_fraude
- hora_transacao
- dia_semana
- data_transacao
- fim_de_semana
- tempo_vida_cartao_dias
- tempo_desde_ultima_transacao_min
- tempo_desde_ultima_transacao_horas
- uso_intervalo_curto
- qtd_transacoes_dia
- qtd_transacoes_24h
- uso_intenso_24h
- linha_repetida
- dispositivo_repetido
- qtd_linhas_distintas_dia
- qtd_dispositivos_distintos_dia
- idade_suspeita
- feriado_bin
- feriado_nao_mapeado
- temp_faixa
- sentido_ida
- clima_adverso
- valor_transacao_faixa
- cartao_qtd_transacoes
- cartao_dias_ativos
- cartao_media_transacoes_por_dia
- cartao_qtd_linhas_distintas
- cartao_qtd_dispositivos_distintos
- cartao_qtd_motoristas_distintos
- cartao_valor_transacao_mean
- cartao_valor_transacao_std
- cartao_pct_integracao
- cartao_pct_feriado
- cartao_pct_intervalo_curto
- periodo_dia
- valor_vs_

<small>***Comentários Letícia:** Neste ponto, consolidei o dataset final garantindo que todas as colunas de rastreio, a variável alvo e as features construídas ao longo do processo estivessem presentes, organizadas e consistentes. Verifiquei o shape final, a quantidade de features e a ordem das colunas para assegurar rastreabilidade, clareza e facilidade de uso nas próximas etapas. Com isso, o dataset ficou validado e pronto para ser exportado e utilizado diretamente na fase de modelagem.*</small>

## 8. Exportação dos dados processados

### 8.1 Export do dataset processado

In [246]:
from pathlib import Path

# Define a raiz do repositório
REPO_ROOT = Path.cwd()

# Caminho para data/processed
DATA_PROCESSED_DIR = REPO_ROOT / "data" / "processed"
DATA_PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Caminho do arquivo final
output_path = DATA_PROCESSED_DIR / "dados_tratados.csv"

# Exporta o dataset processado
df_final.to_csv(output_path, index=False)

print("✔ 8.1 Concluído: dataset processado exportado com sucesso")
print(f"Arquivo salvo em: {output_path}")
print(f"Shape do dataset: {df_final.shape}")

✔ 8.1 Concluído: dataset processado exportado com sucesso
Arquivo salvo em: /content/fraude_bilhetagem/data/processed/dados_tratados.csv
Shape do dataset: (30000, 42)


> Observação: o arquivo `data/processed/dados_tratados.csv` foi versionado no repositório
> para permitir reprodutibilidade e avaliação da etapa de modelagem.


### 8.2 Export do schema de colunas (json)

In [247]:
import json
from pathlib import Path

# Diretório de saída (mesmo padrão do 8.1)
REPO_ROOT = Path("/content/fraude_bilhetagem")
SCHEMA_DIR = REPO_ROOT / "data" / "processed"
SCHEMA_DIR.mkdir(parents=True, exist_ok=True)

schema_path = SCHEMA_DIR / "schema_dados_tratados.json"

# Monta o schema: coluna -> dtype
schema = {
    "n_linhas": int(df_final.shape[0]),
    "n_colunas": int(df_final.shape[1]),
    "colunas": {
        col: str(dtype)
        for col, dtype in df_final.dtypes.items()
    }
}

# Salva em JSON
with open(schema_path, "w", encoding="utf-8") as f:
    json.dump(schema, f, indent=4, ensure_ascii=False)

print("✔ 8.2 Concluído: schema do dataset exportado com sucesso")
print(f"Caminho do arquivo: {schema_path}")

✔ 8.2 Concluído: schema do dataset exportado com sucesso
Caminho do arquivo: /content/fraude_bilhetagem/data/processed/schema_dados_tratados.json


<small>***Comentários Letícia:** Exportei o schema do dataset processado em JSON para documentar, de forma reprodutível, a lista de colunas finais após tratamento e engenharia de features. Com isso, garanti rastreabilidade do formato da base que será consumida na modelagem e reduzi risco de divergência entre notebooks ou execuções em ambientes diferentes.*</small>

### 8.3 Export de dtypes e metadados de processamento

In [252]:
import pandas as pd
from pathlib import Path

# Diretório de saída
REPO_ROOT = Path("/content/fraude_bilhetagem")
DATA_PROCESSED_DIR = REPO_ROOT / "data" / "processed"

# Definições vindas da Etapa 7
colunas_rastreio = ["id_transacao", "id_cartao", "ts_transacao"]
coluna_target = "target_fraude"
colunas_features = [
    c for c in df_final.columns
    if c not in colunas_rastreio + [coluna_target]
]

# Monta dataframe de metadados
meta = pd.DataFrame({
    "coluna": df_final.columns,
    "dtype": df_final.dtypes.astype(str),
    "tipo_logico": [
        "rastreio" if c in colunas_rastreio else
        "target" if c == coluna_target else
        "feature"
        for c in df_final.columns
    ],
    "qtd_valores_unicos": [df_final[c].nunique(dropna=True) for c in df_final.columns],
    "percentual_nulos": [df_final[c].isna().mean() * 100 for c in df_final.columns]
})

# Metadados gerais do dataset
meta_dataset = {
    "qtd_linhas": df_final.shape[0],
    "qtd_colunas": df_final.shape[1],
    "qtd_features": len(colunas_features),
    "qtd_rastreio": len(colunas_rastreio),
    "qtd_target": 1
}

# Salva arquivos
meta_path = DATA_PROCESSED_DIR / "metadados_colunas.csv"
meta_dataset_path = DATA_PROCESSED_DIR / "metadados_dataset.json"

meta.to_csv(meta_path, index=False)
pd.Series(meta_dataset).to_json(meta_dataset_path, indent=2)

print("✔ 8.3 Concluído: metadados exportados com sucesso")
print(f"- Metadados de colunas: {meta_path}")
print(f"- Metadados do dataset: {meta_dataset_path}")

✔ 8.3 Concluído: metadados exportados com sucesso
- Metadados de colunas: /content/fraude_bilhetagem/data/processed/metadados_colunas.csv
- Metadados do dataset: /content/fraude_bilhetagem/data/processed/metadados_dataset.json


<small>***Comentários Letícia:** Optei por exportar metadados separados do dataset processado para documentar de forma clara a estrutura final dos dados e o papel de cada coluna após todas as transformações. Salvei um arquivo específico com informações de colunas e outro com estatísticas gerais do dataset, o que facilita auditoria, reprodutibilidade e entendimento por terceiros. Essa separação também prepara o projeto para futuras etapas de modelagem e versionamento, sem misturar código, dados e documentação técnica.*</small>


### 8.4 Checkpoint de exportação

Nesta subetapa foi realizada a verificação final da exportação dos dados processados e de seus artefatos auxiliares. Confirmou-se que o dataset tratado, o schema de colunas e os metadados do processamento foram corretamente salvos no diretório data/processed, seguindo a estrutura definida do repositório.

Todos os arquivos exportados encontram-se versionados no GitHub, garantindo acesso externo ao avaliador e permitindo a reprodução completa das etapas seguintes do projeto. Com isso, encerra-se a etapa de exportação dos dados, assegurando que o pipeline de tratamento está finalizado e que a modelagem poderá ser conduzida a partir de um conjunto de dados estável, documentado e auditável.

## 9. Insights Finais